# Creating 2 Tables: Sites for Map, Allocations for Table

#### General Approach:
1) Read in csv sheet of data.  Export fields into new database.
2) Remove bad rows and elements.
3) Fix date datatype.
4) Include ID int value for beneficial use.
5) Fix and remove misisng or nan values for allocatoin flow and volume.
6) Remove duplicates based on AllocationNativeID.
7) Create API link for each siteUUID.
8) Export completed df as processed csv.



#### Files Needed
1) dontopen_SiteLJAllow.csv, 
2) dontopen_AlloLFSite.csv

### Sites LF Allow, for creating Site Points, Popups and Legend Colors on Map Only

In [1]:
#Needed Libaraies
import os
import numpy as np
import pandas as pd

#Setting work directory, reading inputs, creating dataframe
workingDir = "C:/Users/rjame/Documents/WSWC Documents/Portal Creation Research"
os.chdir(workingDir)

In [2]:
#Sites LeftJoin Allo Input
SitefileInput = "dontopen_SiteLJAllow.csv"
dfsite = pd.read_csv(SitefileInput)
dfsite

C:\Users\rjame\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SiteID,SiteUUID,SiteTypeCV,Longitude,Latitude,AllocationAmountID,AllocationNativeID,AllocationAmount,AllocationMaximum,IrrigatedAcreage,...,PrimaryUseCategoryCV,AllocationOwner,BeneficialUseCV,Date,WaterSourceID,WaterSourceName,WaterSourceTypeCV,OrganizationID,OrganizationUUID,State
0,5987443,CA_1,NaN,-124.100000,40.963100,3582161.0,A010767,NaN,0.0,0.0,...,Irrigation,BEAU PRE CORP,Domestic,1944-02-14,546831.0,UNSP,Unknown,17.0,CSWRCB,CA
1,5987444,CA_10,NaN,-121.173807,39.875367,3583769.0,A014760,NaN,0.0,0.0,...,Irrigation,RUSSELL WESTOVER JR,Domestic,1952-04-17,546831.0,UNSP,Unknown,17.0,CSWRCB,CA
2,5987445,CA_1000,NaN,-122.716594,41.730083,3587847.0,A021986,NaN,0.0,0.0,...,Irrigation,LAURI HUNNER,Domestic,1964-12-01,546829.0,UNSP (2),Unknown,17.0,CSWRCB,CA
3,5987446,CA_10000,NaN,-120.647606,40.636866,3594308.0,C000897,NaN,0.0,0.0,...,Irrigation,CASIANO LAND & LIVESTOCK COMPANY,Stockwatering,1977-09-07,546844.0,UNST,Unknown,17.0,CSWRCB,CA
4,5987447,CA_10001,NaN,-120.895545,38.450445,3594345.0,C000934,NaN,0.0,0.0,...,Irrigation,ELDEN WAIT,Stockwatering,1977-09-21,546844.0,UNST,Unknown,17.0,CSWRCB,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260871,5768281,WA_9995,headworks gravity flow (or surface water devic...,-120.162283,46.016114,3384138.0,2.05850e+06,0.000000,-999.0,-999.0,...,Irrigation,"JENSEN, SHERMAN C.",Stock water,1974-04-29,462900.0,Unspecified,groundwater,11.0,WSDE,WA
1260872,5768282,WA_9996,well (or ground water device unknown),-120.162290,46.019457,3383038.0,2.05594e+06,0.011140,1.0,-999.0,...,Irrigation,"JENSEN, SHERMAN C",Stock water,1946-10-01,462900.0,Unspecified,groundwater,11.0,WSDE,WA
1260873,5768283,WA_9997,headworks gravity flow (or surface water devic...,-120.122189,46.021328,3395072.0,2.08828e+06,0.020000,0.2,0.0,...,Irrigation,"SEVEN A E RANCHES LLC,",Stock water,1913-01-01,462901.0,Unspecified,surfaceWater,11.0,WSDE,WA
1260874,5768284,WA_9998,well (or ground water device unknown),-120.122641,46.006227,3393054.0,2.07502e+06,0.015596,2.0,-999.0,...,Irrigation,"BROMLEY, C. H.",Domestic general,1947-12-01,462900.0,Unspecified,groundwater,11.0,WSDE,WA


In [3]:
#removeing bad rows of df because they are null for some odd reason
dfsite = dfsite[(dfsite.WaterSourceName != 'test')]
dfsite = dfsite[dfsite['SiteUUID'].notna()]
dfsite = dfsite[dfsite['AllocationPriorityDateID'].notna()]
dfsite = dfsite[dfsite['BeneficialUseCV'].notna()]
dfsite = dfsite[dfsite['Longitude'].notna()]
dfsite = dfsite[dfsite['Latitude'].notna()]
dfsite.reset_index()

,index,SiteID,SiteUUID,SiteTypeCV,Longitude,Latitude,AllocationAmountID,AllocationNativeID,AllocationAmount,AllocationMaximum,...,PrimaryUseCategoryCV,AllocationOwner,BeneficialUseCV,Date,WaterSourceID,WaterSourceName,WaterSourceTypeCV,OrganizationID,OrganizationUUID,State
0,0,5987443,CA_1,NaN,-124.100000,40.963100,3582161.0,A010767,NaN,0.0,...,Irrigation,BEAU PRE CORP,Domestic,1944-02-14,546831.0,UNSP,Unknown,17.0,CSWRCB,CA
1,1,5987444,CA_10,NaN,-121.173807,39.875367,3583769.0,A014760,NaN,0.0,...,Irrigation,RUSSELL WESTOVER JR,Domestic,1952-04-17,546831.0,UNSP,Unknown,17.0,CSWRCB,CA
2,2,5987445,CA_1000,NaN,-122.716594,41.730083,3587847.0,A021986,NaN,0.0,...,Irrigation,LAURI HUNNER,Domestic,1964-12-01,546829.0,UNSP (2),Unknown,17.0,CSWRCB,CA
3,3,5987446,CA_10000,NaN,-120.647606,40.636866,3594308.0,C000897,NaN,0.0,...,Irrigation,CASIANO LAND & LIVESTOCK COMPANY,Stockwatering,1977-09-07,546844.0,UNST,Unknown,17.0,CSWRCB,CA
4,4,5987447,CA_10001,NaN,-120.895545,38.450445,3594345.0,C000934,NaN,0.0,...,Irrigation,ELDEN WAIT,Stockwatering,1977-09-21,546844.0,UNST,Unknown,17.0,CSWRCB,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048418,1260871,5768281,WA_9995,headworks gravity flow (or surface water devic...,-120.162283,46.016114,3384138.0,2.05850e+06,0.000000,-999.0,...,Irrigation,"JENSEN, SHERMAN C.",Stock water,1974-04-29,462900.0,Unspecified,groundwater,11.0,WSDE,WA
1048419,1260872,5768282,WA_9996,well (or ground water device unknown),-120.162290,46.019457,3383038.0,2.05594e+06,0.011140,1.0,...,Irrigation,"JENSEN, SHERMAN C",Stock water,1946-10-01,462900.0,Unspecified,groundwater,11.0,WSDE,WA
1048420,1260873,5768283,WA_9997,headworks gravity flow (or surface water devic...,-120.122189,46.021328,3395072.0,2.08828e+06,0.020000,0.2,...,Irrigation,"SEVEN A E RANCHES LLC,",Stock water,1913-01-01,462901.0,Unspecified,surfaceWater,11.0,WSDE,WA
1048421,1260874,5768284,WA_9998,well (or ground water device unknown),-120.122641,46.006227,3393054.0,2.07502e+06,0.015596,2.0,...,Irrigation,"BROMLEY, C. H.",Domestic general,1947-12-01,462900.0,Unspecified,groundwater,11.0,WSDE,WA


In [4]:
SiteWSWCBenUseDict = {
"0 RELEASE FOR ENVIROMENT" : "Environmental",
"100 Year Flood Flow" : "Flood Control",
"3 STORAGE" : "Storage",
"508-14" : "Other",
"72-12-1 Construction of public works" : "Industrial",
"72-12-1 domestic and livestock watering" : "Domestic",
"72-12-1 domestic one household" : "Domestic",
"72-12-1 livestock watering" : "Livestock",
"72-12-1 multiple domestic households" : "Domestic",
"72-12-1 Prospecting or development of natural resource" : "Environmental",
"72-12-1 Sanitary in conjunction with a commercial use" : "Commercial",
"Administrative" : "Other",
"Aesthetic" : "Other",
"Aesthetic From Storage" : "Other",
"Aesthetic Storage" : "Other",
"Aesthetics" : "Other",
"Agricultural Irrigation" : "Agricultural",
"Agriculture" : "Agricultural",
"AGRICULTURE - WETLAND" : "Agricultural",
"Agriculture other than irrigation" : "Agricultural",
"Agriculture Uses" : "Agricultural",
"Agricutural Spraying" : "Agricultural",
"Air Conditioning or Heating" : "Heating and Cooling",
"All Beneficial Uses" : "Other",
"Anadromous and Resident Fish Habitat (Instream)" : "Fish",
"Aquaculture" : "Aquaculture",
"Aquatic Habitat" : "Instream Flow",
"Aquatic Life Instream" : "Instream Flow",
"As Decreed" : "Other",
"Augmentation" : "Other",
"Augmentation well" : "Environmental",
"Average Annual Flow" : "Environmental",
"Bankfull Flow" : "Environmental",
"Base Flow" : "Environmental",
"BED AND BANKS" : "Environmental",
"Brine production well" : "Environmental",
"Campsite" : "Recreation",
"Cathodic protection well" : "Other",
"Cemetery" : "Industrial",
"Channel Maintenance" : "Environmental",
"Closed file" : "Other",
"Commercial" : "Commercial",
"Commercial & indust" : "Commercial",
"Commercial From Storage" : "Commercial",
"Commercial Storage" : "Commercial",
"Commercial Uses" : "Commercial",
"Community type use - mdwca, private or commercial supplied" : "Commercial",
"Construction" : "Industrial",
"Cooling" : "Heating and Cooling",
"Cooling for indust proces" : "Heating and Cooling",
"Cooling From Storage" : "Heating and Cooling",
"Cooling Storage" : "Heating and Cooling",
"Cranberry" : "Agricultural",
"Cumulative Accretion To River" : "Environmental",
"Cumulative Depletion From River" : "Environmental",
"Dairy" : "Agricultural",
"Dairy Barn Uses" : "Agricultural",
"Dairy operation" : "Industrial",
"Dewatering" : "Other",
"Dewatering well" : "Other",
"Diversion To Storage" : "Other",
"Domestic" : "Domestic",
"Domestic and Livestock" : "Domestic",
"DOMESTIC AND LIVESTOCK & LIVESTOCK" : "Domestic",
"Domestic construction" : "Domestic",
"Domestic Expanded" : "Domestic",
"Domestic From Storage" : "Domestic",
"Domestic general" : "Domestic",
"Domestic Including Lawn and Garden" : "Domestic",
"Domestic multiple" : "Domestic",
"Domestic single" : "Domestic",
"Domestic Storage" : "Domestic",
"Dust Control" : "Environmental",
"Environmental" : "Environmental",
"Environmental quality" : "Environmental",
"Erosion Control" : "Environmental",
"Evaporation" : "Environmental",
"Evaporative" : "Other",
"Exploration" : "Other",
"Export From Basin" : "Other",
"Export From State" : "Other",
"Exports/Imports" : "Other",
"Federal Reserved" : "Other",
"Federal Reserved Use" : "Other",
"Feed pen operation" : "Agricultural",
"Fire" : "Fire",
"Fire Protection" : "Fire",
"Fire Protection (Instream)" : "Fire",
"Fire Protection From Storage" : "Fire",
"Fire Protection Storage" : "Fire",
"Fish & Wildlife" : "Other",
"Fish and game propogation" : "Fish",
"Fish And Wildlife" : "Fish",
"Fish and Wildlife Preservation and Enhancement" : "Fish",
"Fish Culture" : "Fish",
"Fish Habitat" : "Fish",
"Fish Habitat From Storage" : "Fish",
"Fish Habitat Storage" : "Fish",
"Fish Propagation" : "Fish",
"Fish Propagation From Storage" : "Fish",
"Fish Propagation Storage" : "Fish",
"Fish Raceways" : "Fish",
"Fishery" : "Fish",
"Fishery Enhancement (Instream)" : "Fish",
"Flood Control" : "Flood Control",
"Flood Control Storage" : "Flood Control",
"Flow Augmentation" : "Flood Control",
"Flow Through Fish Pond" : "Flood Control",
"Forest Management" : "Other",
"Frost Protection" : "Environmental",
"GAME PRESERVES" : "Environmental",
"Geothermal" : "Power",
"Geo-Thermal (Heating & Cooling)" : "Heating and Cooling",
"Geothermal boreholes" : "Power",
"Geothermal Heating" : "Heating and Cooling",
"Geo-Thermal(Energy Production)" : "Heating and Cooling",
"Greenhouse" : "Environmental",
"Ground Water Recharge" : "Environmental",
"Ground Water Recharge From Storage" : "Environmental",
"Ground Water Recharge Storage" : "Environmental",
"Groundwater Aquifer Storage & Recovery" : "Storage",
"Groundwater Preservation" : "Storage",
"Groundwater Recharge" : "Storage",
"Group Domestic" : "Domestic",
"Harvesting of Cranberries" : "Agricultural",
"Heat Control" : "Heating and Cooling",
"Heat Exchange" : "Heating and Cooling",
"Heat protection for crops" : "Heating and Cooling",
"Heating" : "Heating and Cooling",
"Heating From Storage" : "Heating and Cooling",
"Heating Storage" : "Heating and Cooling",
"Highway" : "Industrial",
"Highway construction" : "Industrial",
"Household Use Only" : "Domestic",
"Human Consumption" : "Domestic",
"Human Consumption and Livestock" : "Domestic",
"HYDROELECTRIC" : "Power",
"HYDROELECTRIC - PRIORITY" : "Power",
"Incidental Power" : "Power",
"Incidental Underground Storage" : "Storage",
"Individual Irrigation" : "Agricultural",
"Industrial" : "Industrial",
"INDUSTRIAL - POWER GENERATION" : "Industrial",
"INDUSTRIAL - SALT WATER" : "Industrial",
"Industrial From Storage" : "Industrial",
"Industrial Storage" : "Industrial",
"Industrial/Manufacturing Uses" : "Industrial",
"Injection" : "Other",
"Institutional" : "Domestic",
"INSTREAM" : "Instream Flow",
"Instream Flow" : "Instream Flow",
"Intentional Underground Storage" : "Storage",
"Irrigated Agriculture " : "Agricultural",
"Irrigated agriculture_ground" : "Agricultural",
"Irrigated agriculture_reuse" : "Agricultural",
"Irrigated agriculture_surface" : "Agricultural",
"Irrigation" : "Agricultural",
"Irrigation and Domestic" : "Agricultural",
"Irrigation and Incidental Underground Storage" : "Storage",
"Irrigation and Livestock" : "Agricultural",
"Irrigation and Storage (an appropriation approved for both uses)" : "Storage",
"Irrigation from Natural Stream" : "Agricultural",
"Irrigation From Storage" : "Industrial",
"Irrigation Livestock and Domestic" : "Agricultural",
"Irrigation of Cranberries" : "Agricultural",
"Irrigation Storage" : "Agricultural",
"Irrigation Unknown" : "Agricultural",
"Irrigation-Carey Act" : "Agricultural",
"Irrigation-DLE" : "Agricultural",
"Laboratory" : "Other",
"Lake Level Maintenance" : "Environmental",
"Lawn And Garden" : "Domestic",
"Livestock" : "Agricultural",
"Livestock " : "Livestock",
"Livestock and Wildlife" : "Agricultural",
"Livestock_ground" : "Agricultural",
"Livestock_reuse" : "Agricultural",
"Livestock_surface" : "Agricultural",
"Log Deck Sprinkling" : "Industrial",
"M & I" : "Municipal",
"Maintain Level of a Lake" : "Recreation",
"Major and Minor Reservoirs" : "Other",
"Manufacturing" : "Industrial",
"Manufacturing_ground" : "Industrial",
"Manufacturing_reuse" : "Industrial",
"Manufacturing_surface" : "Industrial",
"Meat packing plant" : "Industrial",
"Military - military installations" : "Other",
"Milling" : "Industrial",
"Minerals" : "Other",
"Minimum Stream Flow" : "Environmental",
"Minimum Stream Flow From Storage" : "Environmental",
"Minimum Stream Flow Storage" : "Environmental",
"Minimum Streamflow" : "Environmental",
"Mining" : "Mining",
"Mining and Milling" : "Mining",
"Mining From Storage" : "Mining",
"Mining or milling or oil" : "Mining",
"Mining Storage" : "Mining",
"Mining, Milling, and Dewatering" : "Mining",
"Mining_ground" : "Mining",
"Mining_reuse" : "Mining",
"Mining_surface" : "Mining",
"Mitigation" : "Environmental",
"Mitigation By Delivery" : "Flood Control",
"Mitigation By Delivery From Storage" : "Flood Control",
"Mitigation By Delivery Storage" : "Flood Control",
"Mitigation By NonUse" : "Flood Control",
"Mobile home parks" : "Domestic",
"Monitoring well" : "Other",
"MULTI USE" : "Other",
"Multiple Domestic" : "Domestic",
"Multiple Instream Uses" : "Instream Flow",
"Multiple Purpose" : "Other",
"Multiple Use" : "Other",
"Municipal" : "Municipal",
"Municipal - city or county supplied water" : "Municipal",
"Municipal From Storage" : "Municipal",
"Municipal inter-tie system" : "Other",
"Municipal Storage" : "Municipal",
"Municipal Uses" : "Municipal",
"MUNICIPAL/DOMESTIC AND LIVESTOCK" : "Municipal",
"Municipal_ground" : "Municipal",
"Municipal_reuse" : "Municipal",
"Municipal_surface" : "Municipal",
"Navigation" : "Other",
"Net Effect On River" : "Environmental",
"No pre-1907 water right exists on this land" : "Other",
"No Purpose Identified" : "Unknown",
"No right" : "Unknown",
"No use of right or POD" : "Unknown",
"Non 72-12-1 domestic and livestock watering" : "Other",
"Non 72-12-1 domestic one household" : "Other",
"Non 72-12-1 livestock watering" : "Other",
"Non 72-12-1 multiple domestic households" : "Other",
"NON CONSUMPTIVE" : "Other",
"NON-CONSUMPTIVE" : "Other",
"Non-profit organizational use" : "Other",
"Nursery Uses" : "Agricultural",
"Observation" : "Other",
"Observation And Testing" : "Other",
"Oil field maintenance" : "Industrial",
"Oil production" : "Industrial",
"Oil Well Flooding" : "Industrial",
"Other" : "Other",
"OTHER (FISH HATCHERY)" : "Fish",
"Other Purpose" : "Other",
"OTHER SPILLWAY/FLOODGAT" : "Flood Control",
"Parks and Recreation" : "Recreation",
"Petroleum processing plant" : "Industrial",
"Pollution Abatement" : "Instream Flow",
"Pollution Abatement (Instream)" : "Instream Flow",
"Pollution control well" : "Instream Flow",
"Pond Maintenance" : "Municipal",
"Poultry and egg operation" : "Industrial",
"Power" : "Power",
"Power and Incidental Underground Storage" : "Power",
"Power Development" : "Power",
"Power From Storage" : "Power",
"Power Generation" : "Power",
"Power Generation Nonconsumptive" : "Power",
"POWER GENERATION, NONCONSUMPTIVE" : "Power",
"Power Storage" : "Power",
"Practicably Irrigable Acreage" : "Agricultural",
"Primary and Supplemental Irrigation" : "Agricultural",
"PUBLIC PARKS" : "Recreation",
"Public Supply" : "Domestic",
"Public utility" : "Domestic",
"Quasi-Municipal" : "Municipal",
"Quasi-Municipal Uses" : "Other",
"Railway" : "Industrial",
"Raise Dam (for increase in head for power production)" : "Power",
"Ram" : "Industrial",
"Recharge" : "Recharge",
"Recharge For Irrigation" : "Agricultural",
"Recreation" : "Recreation",
"Recreation - beautification" : "Recreation",
"Recreation Fish Wildlife" : "Fish",
"Recreation From Storage" : "Recreation",
"Recreation Storage" : "Recreation",
"Recreational" : "Recreation",
"Restroom" : "Municipal",
"REUSE" : "Other",
"Riparian Habitat (Instream)" : "Instream Flow",
"Riparian Habitat Restoration Project" : "Instream Flow",
"Riparian Maintenance" : "Instream Flow",
"Road Construction" : "Industrial",
"Rural Water" : "Domestic",
"Sale" : "Commercial",
"Sawmill" : "Industrial",
"School use - public, private, parochial, & universities" : "Municipal",
"Secondary recovery of oil" : "Industrial",
"Sediment Control" : "Environmental",
"See Remarks" : "Other",
"Shop" : "Commercial",
"Snow Making" : "Snow Making",
"Stacked water right" : "Other",
"Steam-Electric Power_ground" : "Power",
"Steam-Electric Power_reuse" : "Power",
"Steam-Electric Power_surface" : "Power",
"Stock" : "Livestock",
"Stock water" : "Livestock",
"Stockpond" : "Other",
"STOCKRAISING" : "Livestock",
"Stockwater" : "Livestock",
"Stockwater From Storage" : "Livestock",
"Stockwater Storage" : "Livestock",
"Stockwatering" : "Livestock",
"Storage" : "Storage",
"Storage and Incidental Underground Storage" : "Storage",
"Storage Use-only (irrigation from reservoir on lands not covered by natural flow appropriation)" : "Storage",
"Storage Use-only for irrigation from reservoir on lands not covered by natural flow appropriation AND for Incidental Underground Storage" : "Storage",
"Strategic water reserve" : "Storage",
"Stream augmentation" : "Flood Control",
"Streamflow Maintenance From Storage" : "Storage",
"Streamflow Maintenance Storage" : "Storage",
"Subdivision" : "Domestic",
"Subirrigation" : "Agricultural",
"Supplemental Cooling (an appropriation for water for cooling through a system that has a prior appropriation for cooling)" : "Heating and Cooling",
"Supplemental Flood Harvesting" : "Flood Control",
"Supplemental Irrigation" : "Agricultural",
"Supplemental Irrigation (irrigation from reservoir on lands also covered by natural flow appropriation.)" : "Agricultural",
"Supplemental Irrigation and Incidental Underground Storage" : "Agricultural",
"Supplemental Power (an appropriation for water for power through a system that has a prior appropriation for power)" : "Power",
"Supplemental Storage (an appropriation that has a prior appropriation for storage)" : "Storage",
"Swimming" : "Recreation",
"Temperature Control" : "Heating and Cooling",
"Temporary Transfer to In-Stream Use" : "Storage",
"Test Well" : "Other",
"To be determined" : "Unknown",
"Transmountain Export" : "Other",
"Trust water, Permanent" : "Other",
"Trust water, Temporary" : "Other",
"Unknown" : "Unknown",
"Unspecified" : "Unknown",
"Use in a Mint Still" : "Industrial",
"Use Within a School" : "Municipal",
"Waste Storage" : "Other",
"Water Fowl" : "Wildlife",
"WATER QUALITY" : "Environmental",
"Water Quality Improvement" : "Environmental",
"Water Quality Improvement From Storage" : "Environmental",
"Water Quality Improvement Storage" : "Environmental",
"Wetlands" : "Environmental",
"Wild And Scenic River" : "Wildlife",
"Wildlife" : "Wildlife",
"Wildlife From Storage" : "Wildlife",
"WILDLIFE MANAGEMENT" : "Wildlife",
"Wildlife refuge" : "Wildlife",
"Wildlife Storage" : "Wildlife",
"Wildlife Waterfowl" : "Wildlife",
"WILDLIFE/WATERFOWL" : "Wildlife"}


def assignWSWCBU(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return np.nan
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = SiteWSWCBenUseDict[String1]
        except:
            outList = "Other"
    return outList

dfsite['WSWCBeneficialUse'] = dfsite.apply(lambda row: assignWSWCBU(row['BeneficialUseCV']), axis=1)

In [5]:
#SiteType
SiteTypeCVDic = {"Abandoned" : "Abandoned",
"Aquifer Nnt/Nt Reservation" : "Aquifer",
"Augmentation/Replacement Plan" : "Other",
"canal" : "Canal / Ditch / Stream",
"D/S Limit - Discharge Segment" : "Discharge Point",
"D/S Limit - Diversion Segment" : "Other",
"D/S Limit - Release Segment" : "Other",
"Discharge Point" : "Discharge Point",
"Ditch System" : "Canal / Ditch / Stream",
"ditch" : "Canal / Ditch / Stream",
"Diversion Point - Well" : "Well",
"Diversion Point" : "Diversion Point",
"Diversion" : "Diversion Point",
"drain" : "Drain",
"Effluent" : "Effluent",
"Exchange Plan" : "Exchange Plan",
"Geothermal" : "Other",
"Gravity Other" : "Other",
"Gravity Water-Pump" : "Pump",
"Gravity Well-Pump" : "Well",
"Gravity" : "Gravity",
"ground water collector" : "Groundwater",
"Ground Water" : "Groundwater",
"Groundwater" : "Groundwater",
"headworks gravity flow (or surface water device unknown)" : "Surface Water",
"IBT - Discharge Point" : "Discharge Point",
"IBT - Diversion Point" : "Diversion Point",
"IBT - Off-channel Diversion Pt" : "Diversion Point",
"IBT - Off-channel Reservoir" : "Reservoir",
"IBT - On-channel Reservoir" : "Reservoir",
"IBT - Release Point" : "Other",
"IBT - Return Flow" : "Other",
"irrigation dam" : "Reservoir",
"lake" : "Lake",
"Measuring Point" : "Other",
"Mine" : "Mine",
"Minimum Flow" : "Minimum Flow",
"monitoring well" : "Monitoring Well",
"Off-channel Diversion Point" : "Discharge Point",
"Off-channel Reservoir Complex" : "Reservoir",
"Off-channel Reservoir" : "Reservoir",
"On-channel Reservoir Complex" : "Reservoir",
"On-channel Reservoir" : "Reservoir",
"Other - Rec Use" : "Other",
"Other Ground Water" : "Groundwater",
"Other Surface Water" : "Surface Water",
"Other" : "Other",
"Pipeline" : "Pipeline",
"Point of Rediversion" : "Point of Rediversion",
"Point of Return" : "Point of Return",
"pond" : "Surface Water",
"Power Plant" : "Power Plant",
"Pump" : "Groundwater",
"Reach (Aggregating)" : "Reach",
"Reach" : "Reach",
"Recharge Area Group" : "Recharge Area",
"Recharge Area" : "Recharge Area",
"Release Point" : "Discharge Point",
"reservoir dam" : "Reservoir",
"Reservoir System" : "Reservoir",
"reservoir" : "Reservoir",
"Return Flow Point" : "Point of Return",
"Seep" : "Seep",
"sewage effluent" : "Effluent",
"slough" : "Other",
"spring" : "Spring",
"Storage" : "Storage",
"Stream Gage" : "Other",
"stream" : "Canal / Ditch / Stream",
"sump" : "Sump",
"surface water pump" : "Surface Water",
"Surface Water" : "Surface Water",
"Surface" : "Surface Water",
"Surfacewater_Groundwater" : "Surface & Groundwater",
"U/S Dam" : "Reservoir",
"U/S Limit - Discharge Segment" : "Discharge Point",
"U/S Limit - Diversion Segment" : "Other",
"U/S Limit - Release Segment" : "Other",
"Underground" : "Groundwater",
"Unknown" : "Unknown",
"waste water" : "Effluent",
"Water-Pump Well-Pump" : "Well",
"Water-Pump" : "Pump",
"well (or ground water device unknown)" : "Well",
"Well Field" : "Well",
"Well Group" : "Well",
"well" : "Well",
"Well-Pump Other" : "Pump",
"Well-Pump" : "Well",
"winter runoff" : "winter runoff",
"WWTP Release Point" : "Discharge Point"}
 
def assignSiteType(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "Unknown"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = SiteTypeCVDic[String1]
        except:
            outList = "Unknown"
    return outList

dfsite['SiteTypeCV'] = dfsite.apply(lambda row: assignSiteType(row['SiteTypeCV']), axis=1)

In [6]:
#WaterSourceTypeCV
WaterSourceCVDic = {"Abandoned" : "Other",
"Drain" : "Other",
"Ground Water" : "Groundwater",
"Ground" : "Groundwater",
"Groundwater" : "Groundwater",
"Groundwater/all" : "Groundwater",
"Groundwater/Artesian" : "Groundwater",
"Groundwater/Artesian,Surface Water" : "Surface & Groundwater",
"Groundwater/Dry" : "Groundwater",
"Groundwater/Shallow" : "Groundwater",
"Groundwater/Shallow,Surface Water" : "Surface & Groundwater",
"groundwater/spring" : "Groundwater",
"groundwater/well" : "Groundwater",
"Point of Rediversion" : "Other",
"Point of Return" : "Other",
"reservoir" : "Reservoir / Storage",
"Reuse" : "Other",
"storage" : "Reservoir / Storage",
"Subsurface" : "Other",
"Surface Water" : "Surface Water",
"Surface Water" : "Surface Water",
"Surface" : "Surface Water",
"Surfacewater" : "Surface Water",
"Unknown" : "Unknown",
"Unspecified" : "Unknown"}
 
def assignWaterSourceType(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "Unknown"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = WaterSourceCVDic[String1]
        except:
            outList = "Unknown"
    return outList

dfsite['WaterSourceTypeCV'] = dfsite.apply(lambda row: assignWaterSourceType(row['WaterSourceTypeCV']), axis=1)

In [7]:
#AllocationOwner
AllocationOwnerDic = {
"DREAMWORK MANAGEMENT INC; U.S.-FOREST SERVICE; NEL, ALLEN G." : "US Forest Service",
"FOREST SERVICE LINE SPG" : "US Forest Service",
"FOREST SERVICE SPRING PIPELINE" : "US Forest Service",
"FOREST SERVICE,UNITED STATES" : "US Forest Service",
"FOREST SERVICE,UNITED STATES OF AMERICA" : "US Forest Service",
"Forest Service,United States of America -" : "US Forest Service",
"FOREST SERVICE,USDA" : "US Forest Service",
"MAGDALENA RANGER DISTRICT,US Forest Service" : "US Forest Service",
"N.D. Forest Service" : "US Forest Service",
"Rowser and et al.,Lillian C.,USA Forest Service,Rasmussen,Roger J." : "US Forest Service",
"Rowser and et al.,Lillian C.,USA Forest Service,Rasmusson,Roger J.,Hood,E. Ward & Linnea" : "US Forest Service",
"SANTA FE NATIONAL FOREST,USDA FOREST SERVICE" : "US Forest Service",
"State of Utah Division of Wildlife Resources,USA Forest Service" : "US Forest Service",
"Steed,Newel,USA Forest Service" : "US Forest Service",
"TEXAS A&M FOREST SERVICE" : "US Forest Service",
"U S FOREST SERVICE" : "US Forest Service",
"U S Forest Service" : "US Forest Service",
"U. S. FOREST SERVICE" : "US Forest Service",
"U. S. Forest Service, Lassen National Forest, Hat Creek Ranger District" : "US Forest Service",
"U.S. Forest Service" : "US Forest Service",
"U.S. FOREST SERVICE" : "US Forest Service",
"U.S. FOREST SERVICE" : "US Forest Service",
"U.S. FOREST SERVICE; APPLEGATE RANGER DISTRICT" : "US Forest Service",
"U.S. FOREST SERVICE; BEND RANGER DISTRICT" : "US Forest Service",
"U.S. FOREST SERVICE; COLUMBIA RIVER GORGE SCENIC AREA" : "US Forest Service",
"U.S. FOREST SERVICE; MIDDLE FORK RANGER DISTRICT" : "US Forest Service",
"U.S.-FOREST SERVICE" : "US Forest Service",
"U.S.-FOREST SERVICE; U.S.-AGRICULTURE DEPARTMENT" : "US Forest Service",
"UNITED STATES FOREST SERVICE" : "US Forest Service",
"UNITED STATES FOREST SERVICE" : "US Forest Service",
"United States Of America Acting Through Usda Forest Service" : "US Forest Service",
"UNITED STATES OF AMERICA FOREST SERVICE" : "US Forest Service",
"UNITED STATES OF AMERICA, FOREST SERVICE" : "US Forest Service",
"US Forest Service" : "US Forest Service",
"US Forest Service" : "US Forest Service",
"US Forest Service Mount Baker," : "US Forest Service",
"US Forest Service NW International," : "US Forest Service",
"US Forest Service Snoqualmie," : "US Forest Service",
"US Forest Service Wenatchee," : "US Forest Service",
"US Forest Service," : "US Forest Service",
"US Forest Service, LOS PADRES NATIONAL FOREST" : "US Forest Service",
"USA FOREST SERVICE" : "US Forest Service",
"USA Forest Service" : "US Forest Service",
"USA Forest Service - Dixie National Forest" : "US Forest Service",
"USA Forest Service - Dixie National Service" : "US Forest Service",
"USA Forest Service - Fishlake National Forest" : "US Forest Service",
"USA Forest Service / Fishlake National Forest" : "US Forest Service",
"USA Forest Service,Ackerman,Wilford" : "US Forest Service",
"USA Forest Service,Dixie National Forest,USA FOREST SERVICE" : "US Forest Service",
"USA Forest Service,Gardner,Alan Dean,Gardner,Dean C.,Burgess,C. Judd,Gardner,Larry H." : "US Forest Service",
"USA Forest Service,Heirs of E. J. Hancock" : "US Forest Service",
"USA Forest Service,Jones,Richard N." : "US Forest Service",
"USA Forest Service,USA Forest Service 2" : "US Forest Service",
"USA Forest Service,USA Forestt Service" : "US Forest Service",
"USA Forest Service,USA Forsest Service" : "US Forest Service",
"USA Forest Service." : "US Forest Service",
"USA National Forest 2,USA Forest Service" : "US Forest Service",
"USA USDA FOREST SERVICE" : "US Forest Service",
"USDA FOREST SERVICE" : "US Forest Service",
"USDA Forest Service" : "US Forest Service",
"USDA FOREST SERVICE CIBOLA NF" : "US Forest Service",
"USDA FOREST SERVICE LAKE TAHOE BASIN MANAGEMENT UNIT" : "US Forest Service",
"USDA FOREST SERVICE(CARSON NF)" : "US Forest Service",
"USDA Forest Service," : "US Forest Service",
"USDA FOREST SERVICE, SEQUOIA NATIONAL FOREST" : "US Forest Service",
"USDA Forest Service, USA" : "US Forest Service",
"USDA FOREST SERVICE,CARSON NF" : "US Forest Service",
"USDA FOREST SERVICE; DESCHUTES NATIONAL FOREST" : "US Forest Service",
"USDA FOREST SERVICES" : "US Forest Service",
"USDA, FOREST SERVICE,CARSON NF" : "US Forest Service",
"West Panguitch Irrigation and Reservoir Company,USA Forest Service" : "US Forest Service",
"Department of Energy,United States of America" : "US Department of Energy",
"M-K Ferguson Co. for USA Department of Energy" : "US Department of Energy",
"MK-Ferguson Co. for the USA Department of Energy" : "US Department of Energy",
"U S DEPARTMENT OF ENERGY" : "US Department of Energy",
"U.S. Department of Energy" : "US Department of Energy",
"U.S.-DEPARTMENT OF ENERGY,  YUCCA MOUNTAIN SITE CHARACTERIZATION PROJECT" : "US Department of Energy",
"U.S-DEPARTMENT OF ENERGY, YUCCA MOUNTAIN SITE CHARACTERIZATION PROJECT" : "US Department of Energy",
"UNITED STATES DEPARTMENT OF ENERGY" : "US Department of Energy",
"United States of America,U.S. Department of Energy" : "US Department of Energy",
"US Department of Energy," : "US Department of Energy",
"USA Department of Energy Grand Junction Projects" : "US Department of Energy",
"Bankhead,S. Clair,UT Reclamation Mitigation and Conservation Comm,United States of America Bureau of Reclamation,Corbridge,William B." : "US Bureau Reclamation",
"Bryan, Timothy E & Jennifer,Fletcher, Gary & Carla,Teanaway Associates,,Perkins, Charles J & Catherine,US Bureau Reclamation - Yakima," : "US Bureau Reclamation",
"Bureau Of Reclamation" : "US Bureau Reclamation",
"BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"Bureau Of Reclamation," : "US Bureau Reclamation",
"Bureau of Reclamation, Provo Area Office,United States of America,Richardson,Floyd C.,Richardson,W. Earl,Richardson,Jay V.,Sandy City,Richardson,Duane G." : "US Bureau Reclamation",
"Bureau of Reclamation,The Secretary of the Interior" : "US Bureau Reclamation",
"Bureau of Reclamation,United State of America" : "US Bureau Reclamation",
"BUREAU OF RECLAMATION,UNITED STATES D.O.I." : "US Bureau Reclamation",
"Bureau of Reclamation,United States of America" : "US Bureau Reclamation",
"Bureau of Reclamation,United States of America," : "US Bureau Reclamation",
"Bureau of Reclamation,United States of America,Corbridge,William B." : "US Bureau Reclamation",
"BUREAU OF RECLAMATION; UMATILLA FIELD OFFICE" : "US Bureau Reclamation",
"Heart K Ranch,,US Bureau Reclamation - Yakima," : "US Bureau Reclamation",
"Hirst,Paul and Bonnie,Weber River Water Users Association,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Lambert,Paul,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Morgan County,Weber River Water Users` Assocation,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Provo Area Office,U.S.A. Bureau of Reclamation" : "US Bureau Reclamation",
"Provo Area Office,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Remund,Glen & Unice,United States of America Bureau of Reclamation" : "US Bureau Reclamation",
"Smith,Clarence,Siddoway,Benjamin and Beryl,Vernon,James,Vernon,Francis,Gines,Mark E. and Marianne A.,USA Bureau of Reclamation,Smith,Dick,Siddoway,Alan Siddoway and LouDean,Crandall Creek Irrigation Company" : "US Bureau Reclamation",
"U S BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U. S. DEPT. OF THE INTERIOR,BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S. BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S. Bureau Of Reclamation" : "US Bureau Reclamation",
"U.S. BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S. BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S.A. Bureau of Reclamation" : "US Bureau Reclamation",
"U.S.A. Bureau of Reclamation,Fitzgerald,Jackson R." : "US Bureau Reclamation",
"U.S.-BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"United States Bureau of Reclamation" : "US Bureau Reclamation",
"United States of America Bureau of Reclamation,Spencer Todd and Kelly Colleen Adamson Family Trust" : "US Bureau Reclamation",
"United States of America, Bureau of Reclamation" : "US Bureau Reclamation",
"US BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"US Bureau of Reclamation" : "US Bureau Reclamation",
"US Bureau of Reclamation - Boise," : "US Bureau Reclamation",
"US Bureau Of Reclamation," : "US Bureau Reclamation",
"USA Bureau of Reclamation" : "US Bureau Reclamation",
"USA Bureau of Reclamation -- Provo Area Office" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Boyden,John" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Ivie,James" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Miller,Max" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Weber River Water Users Association,Sanders Properties" : "US Bureau Reclamation",
"UT Reclamation Mitigation and Conservation Comm,United States of America Bureau of Reclamation,Bankhead,S. Clair,Corbridge,William B" : "US Bureau Reclamation",
"Weber River Water Users` Association,Morgan County,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Wilde Revocable Trust,Debra A.,Wilde Land and Livestock Company,Wilde `A` Ditch Company,Faddis,Andrew,Wilde,Henry,Wilson,John,Wilde,Richard & Betty,USA Bureau of Reclamation,Beech,Thomas L.,Wilde Revocable Trust,David Elliot,Wilde,Mary" : "US Bureau Reclamation",
"Wilkinson,Allen,United States of America Bureau of Reclamation" : "US Bureau Reclamation",
"Allen,T. L.,USA Bureau of Reclamation,Eldredge,Edmund,Robinson,Steve B. and Julie Ann,North Chalk Creek Irrigation Company" : "US Bureau Reclamation",
" - Cedar City Field Office,US Bureau of Land Management" : "US Bureau of Land Management",
" - Cedar City Field Office,US Department of Interior, Bureau of Land Management" : "US Bureau of Land Management",
" - Cedar City Field Office,US Department of the Interior, Bureau of Land Management" : "US Bureau of Land Management",
" - Cedar City Field Office,USDI Bureau of Land Management" : "US Bureau of Land Management",
"#NAME?,US Department of Interior, Bureau of Land Management" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"Bureau of Land Management" : "US Bureau of Land Management",
"Bureau of Land Management," : "US Bureau of Land Management",
"Bureau of Land Management, Bakersfield" : "US Bureau of Land Management",
"Bureau of Land Management,Box Elder County" : "US Bureau of Land Management",
"Bureau of Land Management,Davis,Matthew W Davis (II) and Shawnee,Tebbs Living Trust,Richard Ray and Irene H." : "US Bureau of Land Management",
"Bureau of Land Management,Flinspach,Richard and Colleen" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,SOCORRO FIELD OFFICE" : "US Bureau of Land Management",
"Bureau of Land Management,Stuntz Valley Ranch LC" : "US Bureau of Land Management",
"Bureau of Land Management,T Y Cattle Company" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,UNITED STATES" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,UNITED STATES GOVERNMENT" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,UNITED STATES OF AMERICA" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,US DEPARTMENT OF INTERIOR" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,US DEPT LAND MANAGEMENT" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,US DEPT OF INTERIOR" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,USDI" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT; OWYHEE FIELD OFFICE" : "US Bureau of Land Management",
"Carbon County,USA Bureau of Land Management" : "US Bureau of Land Management",
"Cedar City Field Office,USDI Bureau of Land Management;" : "US Bureau of Land Management",
"Chew Livestock Inc,Bureau of Land Management" : "US Bureau of Land Management",
"Department of Interior Bureau of Land Management" : "US Bureau of Land Management",
"Department of Natural Resources,State of Utah,,USA Bureau of Land Management" : "US Bureau of Land Management",
"Division of Wildlife Resources,State of Utah,,USA Bureau of Land Management,Price Field Office" : "US Bureau of Land Management",
"Gemstone Mining Inc.,USA Bureau of Land Management" : "US Bureau of Land Management",
"Grouse Creek Ranch LC,USA Bureau of Land Management" : "US Bureau of Land Management",
"Jordan,W. Clift,USA Bureau of Land Management" : "US Bureau of Land Management",
"Materials Energy Research and Recovery,USA Bureau of Land Management" : "US Bureau of Land Management",
"Moab District USA Bureau of Land Management" : "US Bureau of Land Management",
"Nelson,Rick M.,USA Bureau of Land Management" : "US Bureau of Land Management",
"NORTHERN WASHOE RANCHING MANAGEMENT, LLC; DUCK LAKE RANCH, LLC; U.S.-BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"NORTHERN WASHOE RANCHING MANAGEMENT, LLC; U.S.-BUREAU OF LAND MANAGEMENT; DUCK LAKE RANCH, LLC" : "US Bureau of Land Management",
"Richfield District USA Bureau of Land Management" : "US Bureau of Land Management",
"Saint George Field Office,USDI Bureau of Land Management" : "US Bureau of Land Management",
"Saint George Field Office,USDI Bureau of Land Management;" : "US Bureau of Land Management",
"Skull Valley Company Ltd.,USA Bureau of Land Management" : "US Bureau of Land Management",
"Staker,James Allen,USA Bureau of Land Management" : "US Bureau of Land Management",
"State of Utah Division of Wildlife Resources,USA Bureau of Land Management" : "US Bureau of Land Management",
"Thousand Peaks Ranches, Inc.,USA Bureau of Land Management,Richfield District" : "US Bureau of Land Management",
"U S BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S. Bureau Of Land Management" : "US Bureau of Land Management",
"U.S. BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S. DEPARTMENT OF THE INTERIOR; BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S.-BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S.-BUREAU OF LAND MANAGEMENT; BRIGHT-HOLLAND CORPORATION" : "US Bureau of Land Management",
"U.S.-BUREAU OF LAND MANAGEMENT; WHITE PINE LUMBER CO." : "US Bureau of Land Management",
"U.S.D.I., Bureau of Land Management" : "US Bureau of Land Management",
"United States Of America Acting Through The Department Of Interior Bureau Of Land Management" : "US Bureau of Land Management",
"United States of America Bureau of Land Management,Fillmore District" : "US Bureau of Land Management",
"UNITED STATES OF AMERICA, DEPARTMENT OF THE INTERIOR, BUREAU OF LAND MANAGEMENT; WULFENSTEIN TRUST R-501; SPRING MOUNTAIN MOTORSPORTS RANCH; PAHRUMP GATEWAY UTILITY; SPRING MOUNTAIN RACEWAY LLC" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT / BARSTOW FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT / PALM SPRINGS FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT, ALTURAS FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT, RIDGECREST FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT, UKIAH FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT/ ARCATA FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT/NEEDLES FIELD OFFICE" : "US Bureau of Land Management",
"US DEPT OF THE INTERIOR,BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"USA Bureau of Land Management" : "US Bureau of Land Management",
"USA Bureau of Land Management - Filmore Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management (Richfield District)" : "US Bureau of Land Management",
"USA Bureau of Land Management Vernal District" : "US Bureau of Land Management",
"USA Bureau of Land Management, Richfield District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Cedar City District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Fillmore District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Fillmore Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management,Kanab District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Kanab Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management,Moab District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Monticello  District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Monticello District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Ogden District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Price District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Price Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management,Richfield District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Vernal District" : "US Bureau of Land Management",
"USDI BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"USDI BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"Utah School and Institutional Trust Lands Admin.,USA Bureau of Land Management" : "US Bureau of Land Management",
"West Panguitch Irrigation and Reservoir Company,State of Utah Board of Water Resources,USA Bureau of Land Management" : "US Bureau of Land Management",
"WHITE PINE LUMBER CO; U.S.-BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"ARMY CAMP SPRING" : "US Army",
"CORPS OF ENGRS. ABIQUIU DAM,US ARMY ENGR DIST ALBQ." : "US Army",
"Cummins, Dale F,US Army Corps Of Engineers," : "US Army",
"Dept Of The Army Corps Of Engineers," : "US Army",
"LONGHORN ARMY AMMUNITION PLANT,U.S. DEPARTMENT OF THE INTERIOR FISH AND WILDLIFE SERVICE" : "US Army",
"U S ARMY CORPS OF ENGINEERS" : "US Army",
"U S Army Corps of Engineers" : "US Army",
"U S Army, Department of" : "US Army",
"U.S Army Corps Of Engineers" : "US Army",
"U.S. ARMY CORP OF ENGINEERS" : "US Army",
"U.S. Army Corps Of Engineers" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS SEATTLE" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS WALLA WALLA" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS, WALLA WALLA DISTRICT" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS; PORTLAND DISTRICT" : "US Army",
"U.S. ARMY CORPS OF ENGINEERSSEATTLE" : "US Army",
"U.S. ARMY CORPS OF ENGINEERSWALLA WALLA" : "US Army",
"U.S. Army Dugway Proving Ground" : "US Army",
"U.S. ARMY WHITE SANDS MISSILE" : "US Army",
"U.S. ARMY-WHITE SANDS MISSILE" : "US Army",
"UNITED STATES ARMY CORPS OF ENGINEERS" : "US Army",
"US ARMY" : "US Army",
"Us Army Corps Of Engineers" : "US Army",
"US ARMY CORPS OF ENGINEERS" : "US Army",
"US Army Corps Of Engineers," : "US Army",
"US ARMY CORPS OF ENGINGEERS" : "US Army",
"US DEPARTMENT OF THE ARMY" : "US Army",
"USA Army Dugway Proving Ground" : "US Army",
"USA Army Dugway Proving Grounds" : "US Army",
"USA Army Engineer District Corps of Engineers" : "US Army",
"USA Army for Dugway Proving Ground" : "US Army",
"USA Army Ogden Sub-Depot of Tooele Ordinance Depot" : "US Army",
"USA Army Tooele Army Depot South" : "US Army",
"USA Department of the Army" : "US Army",
"USA Department of the Army - Dugway Proving Ground" : "US Army",
"USA Department of the Army - Tooele Army Depot" : "US Army",
"USA Department of the Army Arsenal" : "US Army",
"USARMY Corp Of Engineers," : "US Army",
"USARMY Corps Engineers Columbia Cnty," : "US Army",
"USARMY Corps Engineers Seattle," : "US Army",
"USARMY Corps Engineers Walla Walla," : "US Army",
"USARMY Corps Engineers," : "US Army",
"USARMY Corps Engineers,,USDA Agricultural Research Service WRPIS," : "US Army",
"WA Fish & Wildlife Dept - ERO,,USARMY Corps Engineers," : "US Army",
"Camelot International Resort Corporation,United States of America" : "United States of America",
"Christensen,Mark B.,Fish and Wildlife Service,United States of America" : "United States of America",
"CIBOLA NATIONAL FOREST,UNITED STATES OF AMERICA," : "United States of America",
"Ekker,Sheena,Utah School and Institutional Trust Lands Admin.,Hodges,Jimmy,McArthur,Ty,United States of America" : "United States of America",
"Fish and Wildlife Service,United States of America" : "United States of America",
"Fish and Wildlife Service,United States of America," : "United States of America",
"INTERMOUNTAIN LAND CO.; MOONROCKS GATEWAY LLC; UNITED STATES OF AMERICA" : "United States of America",
"LAMB, SHELDON W. & ISABEL H.; UNITED STATES OF AMERICA" : "United States of America",
"Llewellyn,Robert W.,United States of America" : "United States of America",
"R. CLYDE HIIBEL FAMILY TRUST; UNITED STATES OF AMERICA" : "United States of America",
"The United States of America" : "United States of America",
"THE UNITED STATES OF AMERICA IN TRUST FOR THE PYRAMID LAKE PAIUTE TRIBE; GARAVANTA LAND AND LIVESTOCK COMPANY SUCCESSOR TO J. GARAVANTA" : "United States of America",
"THOMAS, DARRELL E.; THOMAS, BEVERLY J.; UNITED STATES OF AMERICA" : "United States of America",
"United States Of America" : "United States of America",
"UNITED STATES OF AMERICA" : "United States of America",
"UNITED STATES OF AMERICA" : "United States of America",
"United States of America" : "United States of America",
"UNITED STATES OF AMERICA (BLM)" : "United States of America",
"United States Of America Acting Through" : "United States of America",
"United States of America in trust for Raymond Murray,Burton,John Ira" : "United States of America",
"UNITED STATES OF AMERICA IN TRUST FOR SUCH SHOSHONE INDIANS SOUTHERN NEVADA" : "United States of America",
"UNITED STATES OF AMERICA IN TRUST FOR THE WASHOE TRIBE OF INDIANS" : "United States of America",
"UNITED STATES OF AMERICA IN TRUST FOR YOMBA TRIBE OF SHOSHONE INDIANS" : "United States of America",
"UNITED STATES OF AMERICA NAS" : "United States of America",
"UNITED STATES OF AMERICA NAVAL AIR STATION FALLON" : "United States of America",
"United States Of America Trustee" : "United States of America",
"United States Of America," : "United States of America",
"UNITED STATES OF AMERICA,  C/O COMMANDER AND DISTRICT ENGINEER,   UNITED STATES ARMY ENGINEER  DISTRICT, LOUISVILLE" : "United States of America",
"United States of America, Utah" : "United States of America",
"United States of America,BBRD, LC,Ottesen,Nellie" : "United States of America",
"United States of America,Owen Probst and Rosetta B. Probst Trust" : "United States of America",
"United States of America,The,Conger,Wm. B. and Florance H" : "United States of America",
"United States of America,Virgin River Land Preservation Inc." : "United States of America",
"UNITED STATES OF AMERICA; STREETS; UNITED STATES OF AMERICA IN TRUST FOR THE RENO SPARKS INDIAN COLONY; NEVADA-DEPARTMENT OF TRANSPORTATION; SIERRA PACIFIC POWER COMPANY; RENO SPARKS INDIAN COLONY; RENO LODGE #13 F AND A M" : "United States of America",
"United States of American" : "United States of America",
"USA-United States Of America" : "United States of America",
"Utah Reclamation Mitigation and Conservation Commission,United States Of America" : "United States of America",
"WASHOE COUNTY; PRATT, TOM & LANNETTE; UNITED STATES OF AMERICA" : "United States of America",
"WS-BUCKLAND, LLC; UNITED STATES OF AMERICA" : "United States of America"}
 
def assignAllocationOwner(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "Other"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = AllocationOwnerDic[String1]
        except:
            outList = "Other"
    return outList

dfsite['AllocationOwner'] = dfsite.apply(lambda row: assignAllocationOwner(row['AllocationOwner']), axis=1)

In [8]:
#Color for WBenUse for the Site Map per point
#Mapdeck uses a build color pacakge called 'colourvalues library', which does use HEX color code, but has to end in an "FF".
WSWCBenUseColorDict = { 
"Agricultural" : "#006400FF",
"Aquaculture" : "#9ACD32FF",
"Commercial" : "#FFFF00FF",
"Domestic" : "#0000FFFF",
"Environmental" : "#32CD32FF",
"Fire" : "#FF4500FF",
"Fish" : "#9370DBFF",
"Flood Control" : "#00FFFFFF",
"Heating and Cooling" : "#FF69B4FF",
"Industrial" : "#800080FF",
"Instream Flow" : "#00BFFFFF",
"Livestock" : "#FFD700FF",
"Mining" : "#A52A2AFF",
"Municipal" : "#4B0082FF",
"Power" : "#FFA500FF",
"Recharge" : "#D2691EFF",
"Recreation" : "#FFC0CBFF",
"Snow Making" : "#F0FFF0FF",
"Storage" : "#F5DEB3FF",
"Wildlife" : "#FF0000FF",
"Other" : "#808080FF",
"Unknown" : "#D3D3D3FF"}
 
def assignWSWCBUColor(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "#D3D3D3FF"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = WSWCBenUseColorDict[String1]
        except:
            outList = "#D3D3D3FF"
    return outList

dfsite['WBenUseColor'] = dfsite.apply(lambda row: assignWSWCBUColor(row['WSWCBeneficialUse']), axis=1)

In [9]:
#Color for SiteType for the Site Map per point
#Mapdeck uses a build color pacakge called 'colourvalues library', which does use HEX color code, but has to end in an "FF".
WSWCSiteTypeColorDict = {
"Abandoned" : "#FFC0CBFF",
"Aquifer" : "#FF1493FF",
"Canal / Ditch / Stream" : "#191970FF",
"Discharge Point" : "#EE82EEFF",
"Diversion Point" : "#BA55D3FF",
"Drain" : "#8A2BE2FF",
"Effluent" : "#800080FF",
"Exchange Plan" : "#483D8BFF",
"Gravity" : "#4B0082FF",
"Groundwater" : "#FFA07AFF",
"Lake" : "#F08080FF",
"Mine" : "#FF0000FF",
"Minimum Flow" : "#FFA500FF",
"Monitoring Well" : "#FF6347FF",
"Other" : "#FFFF00FF",
"Pipeline" : "#FAFAD2FF",
"Point of Rediversion" : "#EEE8AAFF",
"Point of Return" : "#ADFF2FFF",
"Power Plant" : "#32CD32FF",
"Pump" : "#00FF7FFF",
"Reach" : "#228B22FF",
"Recharge Area" : "#9ACD32FF",
"Reservoir" : "#800000FF",
"Seep" : "#20B2AAFF",
"Spring" : "#00FFFFFF",
"Storage" : "#AFEEEEFF",
"Sump" : "#48D1CCFF",
"Surface & Groundwater" : "#808000FF",
"Surface Water" : "#4682B4FF",
"Unknown" : "#D3D3D3FF",
"Well" : "#6495EDFF",
"winter runoff" : "#1E90FFFF"}
 
def assignWSWCSiteTypeColor(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "#D3D3D3FF"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = WSWCSiteTypeColorDict[String1]
        except:
            outList = "#D3D3D3FF"
    return outList

dfsite['SiteTypeColor'] = dfsite.apply(lambda row: assignWSWCSiteTypeColor(row['SiteTypeCV']), axis=1)

In [10]:
#Color for WaterSourceType for the Site Map per point
#Mapdeck uses a build color pacakge called 'colourvalues library', which does use HEX color code, but has to end in an "FF".
WSWCWaterSourceTypeColorDict = {
"Groundwater" : "#008000FF",
"Other" : "#FFA500FF",
"Reservoir / Storage" : "#FFFF00FF",
"Surface & Groundwater" : "#800080FF",
"Surface Water" : "#FF4500FF",
"Unknown" : "#0000FFFF"}
 
def assignWSWCWaterSourceTypeColor(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "#D3D3D3FF"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = WSWCWaterSourceTypeColorDict[String1]
        except:
            outList = "#D3D3D3FF"
    return outList

dfsite['WaterSourceTypeColor'] = dfsite.apply(lambda row: assignWSWCWaterSourceTypeColor(row['WaterSourceTypeCV']), axis=1)

In [11]:
#Color for Owner for the Site Map per point
#Mapdeck uses a build color pacakge called 'colourvalues library', which does use HEX color code, but has to end in an "FF".
WSWCOwnerColorDict = { 
"US Forest Service" : "#006400FF",
"US Department of Energy" : "#FFA500FF",
"US Bureau Reclamation" : "#FFFF00FF",
"US Bureau of Land Management" : "#800080FF",
"US Army" : "#FF4500FF",
"United States of America" : "#0000FFFF",
"Other" : "#D3D3D3FF"}
 
def assignWSWCOwnerColor(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "#D3D3D3FF"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = WSWCOwnerColorDict[String1]
        except:
            outList = "#D3D3D3FF"
    return outList

dfsite['OwnerColor'] = dfsite.apply(lambda row: assignWSWCOwnerColor(row['AllocationOwner']), axis=1)

In [12]:
#Sort by SiteUUID and BenUse (this way Agriculture should show up first in the below list)
dfsite = dfsite.sort_values(by=['SiteUUID', 'WSWCBeneficialUse'], ascending=True).reset_index()
dfsite

,index,SiteID,SiteUUID,SiteTypeCV,Longitude,Latitude,AllocationAmountID,AllocationNativeID,AllocationAmount,AllocationMaximum,...,WaterSourceName,WaterSourceTypeCV,OrganizationID,OrganizationUUID,State,WSWCBeneficialUse,WBenUseColor,SiteTypeColor,WaterSourceTypeColor,OwnerColor
0,0,5987443,CA_1,Unknown,-124.100000,40.963100,3582161.0,A010767,NaN,0.0,...,UNSP,Unknown,17.0,CSWRCB,CA,Domestic,#0000FFFF,#D3D3D3FF,#0000FFFF,#D3D3D3FF
1,1,5987444,CA_10,Unknown,-121.173807,39.875367,3583769.0,A014760,NaN,0.0,...,UNSP,Unknown,17.0,CSWRCB,CA,Domestic,#0000FFFF,#D3D3D3FF,#0000FFFF,#D3D3D3FF
2,2,5987445,CA_1000,Unknown,-122.716594,41.730083,3587847.0,A021986,NaN,0.0,...,UNSP (2),Unknown,17.0,CSWRCB,CA,Domestic,#0000FFFF,#D3D3D3FF,#0000FFFF,#D3D3D3FF
3,3,5987446,CA_10000,Unknown,-120.647606,40.636866,3594308.0,C000897,NaN,0.0,...,UNST,Unknown,17.0,CSWRCB,CA,Livestock,#FFD700FF,#D3D3D3FF,#0000FFFF,#D3D3D3FF
4,4,5987447,CA_10001,Unknown,-120.895545,38.450445,3594345.0,C000934,NaN,0.0,...,UNST,Unknown,17.0,CSWRCB,CA,Livestock,#FFD700FF,#D3D3D3FF,#0000FFFF,#D3D3D3FF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048418,1260871,5768281,WA_9995,Surface Water,-120.162283,46.016114,3384138.0,2.05850e+06,0.000000,-999.0,...,Unspecified,Unknown,11.0,WSDE,WA,Livestock,#FFD700FF,#4682B4FF,#0000FFFF,#D3D3D3FF
1048419,1260872,5768282,WA_9996,Well,-120.162290,46.019457,3383038.0,2.05594e+06,0.011140,1.0,...,Unspecified,Unknown,11.0,WSDE,WA,Livestock,#FFD700FF,#6495EDFF,#0000FFFF,#D3D3D3FF
1048420,1260873,5768283,WA_9997,Surface Water,-120.122189,46.021328,3395072.0,2.08828e+06,0.020000,0.2,...,Unspecified,Unknown,11.0,WSDE,WA,Livestock,#FFD700FF,#4682B4FF,#0000FFFF,#D3D3D3FF
1048421,1260874,5768284,WA_9998,Well,-120.122641,46.006227,3393054.0,2.07502e+06,0.015596,2.0,...,Unspecified,Unknown,11.0,WSDE,WA,Domestic,#0000FFFF,#6495EDFF,#0000FFFF,#D3D3D3FF


In [13]:
## Create Sites Dataframe - with attached Allo and Benuse info ##
########################################
outdfSite = pd.DataFrame()

#The Columns
outdfSite['SiteUUID'] = dfsite['SiteUUID'].astype(str) 
outdfSite['Lat'] = dfsite['Latitude'].astype(float) 
outdfSite['Long'] = dfsite['Longitude'].astype(float)
outdfSite['WBenUse'] = dfsite['WSWCBeneficialUse'].astype(str)
outdfSite['WBenUseColor'] = dfsite['WBenUseColor'].astype(str)
outdfSite['SiteTypeColor'] = dfsite['SiteTypeColor'].astype(str)
outdfSite['WaterSourceTypeColor'] = dfsite['WaterSourceTypeColor'].astype(str)
outdfSite['OwnerColor'] = dfsite['OwnerColor'].astype(str)


#Things we don't actually need for the site csv, but will need for the allowcation.
# outdfSite['SiteTypeCV'] = dfsite['SiteTypeCV'].astype(str)
# outdfSite['WaterSourceTypeCV'] = dfsite['WaterSourceTypeCV'].astype(str)
# outdfSite['AllocationOwner'] = dfsite['AllocationOwner'].astype(str)

#Drop Duplicates - SiteUUID
outdfSite = outdfSite.drop_duplicates(['SiteUUID'], keep="first").reset_index(drop=True)

outdfSite

,SiteUUID,Lat,Long,WBenUse,WBenUseColor,SiteTypeColor,WaterSourceTypeColor,OwnerColor
0,CA_1,40.963100,-124.100000,Domestic,#0000FFFF,#D3D3D3FF,#0000FFFF,#D3D3D3FF
1,CA_10,39.875367,-121.173807,Domestic,#0000FFFF,#D3D3D3FF,#0000FFFF,#D3D3D3FF
2,CA_1000,41.730083,-122.716594,Domestic,#0000FFFF,#D3D3D3FF,#0000FFFF,#D3D3D3FF
3,CA_10000,40.636866,-120.647606,Livestock,#FFD700FF,#D3D3D3FF,#0000FFFF,#D3D3D3FF
4,CA_10001,38.450445,-120.895545,Livestock,#FFD700FF,#D3D3D3FF,#0000FFFF,#D3D3D3FF
...,...,...,...,...,...,...,...,...
744196,WA_9995,46.016114,-120.162283,Livestock,#FFD700FF,#4682B4FF,#0000FFFF,#D3D3D3FF
744197,WA_9996,46.019457,-120.162290,Livestock,#FFD700FF,#6495EDFF,#0000FFFF,#D3D3D3FF
744198,WA_9997,46.021328,-120.122189,Livestock,#FFD700FF,#4682B4FF,#0000FFFF,#D3D3D3FF
744199,WA_9998,46.006227,-120.122641,Domestic,#0000FFFF,#6495EDFF,#0000FFFF,#D3D3D3FF


In [14]:
#Exporting Outbound DataFrame to working csv file.
outdfSite.to_csv('P_SiteLJAllow.csv', index=False)  # The output

# Allo LF Site, for filtering and Table Return Only

In [15]:
#Needed Libaraies
import os
import numpy as np
import pandas as pd

#Setting work directory, reading inputs, creating dataframe
workingDir = "C:/Users/rjame/Documents/WSWC Documents/Portal Creation Research"
os.chdir(workingDir)

In [16]:
#Allo LeftJoin Sites Input
AllofileInput = "dontopen_AllowLJSite.csv"
dfallo = pd.read_csv(AllofileInput)
dfallo

C:\Users\rjame\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,AllocationAmountID,AllocationNativeID,AllocationAmount,AllocationMaximum,IrrigatedAcreage,AllocationPriorityDateID,PrimaryUseCategoryCV,AllocationOwner,BeneficialUseCV,Date,...,SiteUUID,SiteTypeCV,Longitude,Latitude,WaterSourceID,WaterSourceName,WaterSourceTypeCV,OrganizationID,OrganizationUUID,State
0,3819551,00-126,0.000000,16862.4,NaN,9015,Irrigation,NaN,Unknown,1970-01-01,...,NaN,NaN,NaN,NaN,386678,Utah Lake and Jordan River,Abandoned,3,UTDWRE,UT
1,3819572,01-1000,0.000000,0.0,NaN,9015,Irrigation,"USA Bureau of Land Management,Moab District",Other,1970-01-01,...,NaN,NaN,NaN,NaN,395597,Diamond Canyon Creek,Unknown,3,UTDWRE,UT
2,3819572,01-1000,0.000000,0.0,NaN,9015,Irrigation,"USA Bureau of Land Management,Moab District",Stockwatering,1970-01-01,...,NaN,NaN,NaN,NaN,395597,Diamond Canyon Creek,Unknown,3,UTDWRE,UT
3,3819573,01-1001,0.000000,0.0,NaN,9015,Irrigation,"USA Bureau of Land Management,Moab District",Other,1970-01-01,...,NaN,NaN,NaN,NaN,395597,Diamond Canyon Creek,Unknown,3,UTDWRE,UT
4,3819573,01-1001,0.000000,0.0,NaN,9015,Irrigation,"USA Bureau of Land Management,Moab District",Stockwatering,1970-01-01,...,NaN,NaN,NaN,NaN,395597,Diamond Canyon Creek,Unknown,3,UTDWRE,UT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109737,3384138,2.05850e+06,0.000000,-999.0,-999.0,12754,Irrigation,"JENSEN, SHERMAN C.",Stock water,1974-04-29,...,WA_9995,headworks gravity flow (or surface water devic...,-120.162283,46.016114,462900,Unspecified,groundwater,11,WSDE,WA
1109738,3383038,2.05594e+06,0.011140,1.0,-999.0,7776,Irrigation,"JENSEN, SHERMAN C",Stock water,1946-10-01,...,WA_9996,well (or ground water device unknown),-120.162290,46.019457,462900,Unspecified,groundwater,11,WSDE,WA
1109739,3395072,2.08828e+06,0.020000,0.2,0.0,5707,Irrigation,"SEVEN A E RANCHES LLC,",Stock water,1913-01-01,...,WA_9997,headworks gravity flow (or surface water devic...,-120.122189,46.021328,462901,Unspecified,surfaceWater,11,WSDE,WA
1109740,3393054,2.07502e+06,0.015596,2.0,-999.0,20339,Irrigation,"BROMLEY, C. H.",Domestic general,1947-12-01,...,WA_9998,well (or ground water device unknown),-120.122641,46.006227,462900,Unspecified,groundwater,11,WSDE,WA


In [17]:
#removeing bad rows of df because they are null for some odd reason
dfallo = dfallo[(dfallo.WaterSourceName != 'test')]
dfallo = dfallo[dfallo['SiteUUID'].notna()]
dfallo = dfallo[dfallo['AllocationPriorityDateID'].notna()]
dfallo = dfallo[dfallo['BeneficialUseCV'].notna()]
dfallo = dfallo[dfallo['Longitude'].notna()]
dfallo = dfallo[dfallo['Latitude'].notna()]
dfallo.reset_index()

,index,AllocationAmountID,AllocationNativeID,AllocationAmount,AllocationMaximum,IrrigatedAcreage,AllocationPriorityDateID,PrimaryUseCategoryCV,AllocationOwner,BeneficialUseCV,...,SiteUUID,SiteTypeCV,Longitude,Latitude,WaterSourceID,WaterSourceName,WaterSourceTypeCV,OrganizationID,OrganizationUUID,State
0,57900,3582161,A010767,NaN,0.0,0.0,33567,Irrigation,BEAU PRE CORP,Domestic,...,CA_1,NaN,-124.100000,40.963100,546831,UNSP,Unknown,17,CSWRCB,CA
1,57901,3583769,A014760,NaN,0.0,0.0,22293,Irrigation,RUSSELL WESTOVER JR,Domestic,...,CA_10,NaN,-121.173807,39.875367,546831,UNSP,Unknown,17,CSWRCB,CA
2,57902,3587847,A021986,NaN,0.0,0.0,2282,Irrigation,LAURI HUNNER,Domestic,...,CA_1000,NaN,-122.716594,41.730083,546829,UNSP (2),Unknown,17,CSWRCB,CA
3,57903,3594308,C000897,NaN,0.0,0.0,11225,Irrigation,CASIANO LAND & LIVESTOCK COMPANY,Stockwatering,...,CA_10000,NaN,-120.647606,40.636866,546844,UNST,Unknown,17,CSWRCB,CA
4,57904,3594345,C000934,NaN,0.0,0.0,10198,Irrigation,ELDEN WAIT,Stockwatering,...,CA_10001,NaN,-120.895545,38.450445,546844,UNST,Unknown,17,CSWRCB,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048418,1109737,3384138,2.05850e+06,0.000000,-999.0,-999.0,12754,Irrigation,"JENSEN, SHERMAN C.",Stock water,...,WA_9995,headworks gravity flow (or surface water devic...,-120.162283,46.016114,462900,Unspecified,groundwater,11,WSDE,WA
1048419,1109738,3383038,2.05594e+06,0.011140,1.0,-999.0,7776,Irrigation,"JENSEN, SHERMAN C",Stock water,...,WA_9996,well (or ground water device unknown),-120.162290,46.019457,462900,Unspecified,groundwater,11,WSDE,WA
1048420,1109739,3395072,2.08828e+06,0.020000,0.2,0.0,5707,Irrigation,"SEVEN A E RANCHES LLC,",Stock water,...,WA_9997,headworks gravity flow (or surface water devic...,-120.122189,46.021328,462901,Unspecified,surfaceWater,11,WSDE,WA
1048421,1109740,3393054,2.07502e+06,0.015596,2.0,-999.0,20339,Irrigation,"BROMLEY, C. H.",Domestic general,...,WA_9998,well (or ground water device unknown),-120.122641,46.006227,462900,Unspecified,groundwater,11,WSDE,WA


In [18]:
#changing data type of 'Date' to datatype, then changing format of 'Date' to yyyy-mm-dd
dfallo['Date'] = pd.to_datetime(dfallo['Date'], errors = 'coerce')
dfallo['Date'] = pd.to_datetime(dfallo["Date"].dt.strftime('%m/%d/%Ym'))

In [19]:
#Removing NaN, and missing (999) values from AllocaitonAmount, AllocaitonMaxium, IrrigatedAcerage.
dfallo['AllocationAmount'] = dfallo['AllocationAmount'].fillna(0)
dfallo['AllocationAmount'] = dfallo['AllocationAmount'].replace(999, 0)

dfallo['AllocationMaximum'] = dfallo['AllocationMaximum'].fillna(0)
dfallo['AllocationMaximum'] = dfallo['AllocationMaximum'].replace(999, 0)

dfallo['IrrigatedAcreage'] = dfallo['IrrigatedAcreage'].fillna(0)
dfallo['IrrigatedAcreage'] = dfallo['IrrigatedAcreage'].replace(999, 0)

In [20]:
alloWSWCBenUseDict = {
"0 RELEASE FOR ENVIROMENT" : "Environmental",
"100 Year Flood Flow" : "Flood Control",
"3 STORAGE" : "Storage",
"508-14" : "Other",
"72-12-1 Construction of public works" : "Industrial",
"72-12-1 domestic and livestock watering" : "Domestic",
"72-12-1 domestic one household" : "Domestic",
"72-12-1 livestock watering" : "Livestock",
"72-12-1 multiple domestic households" : "Domestic",
"72-12-1 Prospecting or development of natural resource" : "Environmental",
"72-12-1 Sanitary in conjunction with a commercial use" : "Commercial",
"Administrative" : "Other",
"Aesthetic" : "Other",
"Aesthetic From Storage" : "Other",
"Aesthetic Storage" : "Other",
"Aesthetics" : "Other",
"Agricultural Irrigation" : "Agricultural",
"Agriculture" : "Agricultural",
"AGRICULTURE - WETLAND" : "Agricultural",
"Agriculture other than irrigation" : "Agricultural",
"Agriculture Uses" : "Agricultural",
"Agricutural Spraying" : "Agricultural",
"Air Conditioning or Heating" : "Heating and Cooling",
"All Beneficial Uses" : "Other",
"Anadromous and Resident Fish Habitat (Instream)" : "Fish",
"Aquaculture" : "Aquaculture",
"Aquatic Habitat" : "Instream Flow",
"Aquatic Life Instream" : "Instream Flow",
"As Decreed" : "Other",
"Augmentation" : "Other",
"Augmentation well" : "Environmental",
"Average Annual Flow" : "Environmental",
"Bankfull Flow" : "Environmental",
"Base Flow" : "Environmental",
"BED AND BANKS" : "Environmental",
"Brine production well" : "Environmental",
"Campsite" : "Recreation",
"Cathodic protection well" : "Other",
"Cemetery" : "Industrial",
"Channel Maintenance" : "Environmental",
"Closed file" : "Other",
"Commercial" : "Commercial",
"Commercial & indust" : "Commercial",
"Commercial From Storage" : "Commercial",
"Commercial Storage" : "Commercial",
"Commercial Uses" : "Commercial",
"Community type use - mdwca, private or commercial supplied" : "Commercial",
"Construction" : "Industrial",
"Cooling" : "Heating and Cooling",
"Cooling for indust proces" : "Heating and Cooling",
"Cooling From Storage" : "Heating and Cooling",
"Cooling Storage" : "Heating and Cooling",
"Cranberry" : "Agricultural",
"Cumulative Accretion To River" : "Environmental",
"Cumulative Depletion From River" : "Environmental",
"Dairy" : "Agricultural",
"Dairy Barn Uses" : "Agricultural",
"Dairy operation" : "Industrial",
"Dewatering" : "Other",
"Dewatering well" : "Other",
"Diversion To Storage" : "Other",
"Domestic" : "Domestic",
"Domestic and Livestock" : "Domestic",
"DOMESTIC AND LIVESTOCK & LIVESTOCK" : "Domestic",
"Domestic construction" : "Domestic",
"Domestic Expanded" : "Domestic",
"Domestic From Storage" : "Domestic",
"Domestic general" : "Domestic",
"Domestic Including Lawn and Garden" : "Domestic",
"Domestic multiple" : "Domestic",
"Domestic single" : "Domestic",
"Domestic Storage" : "Domestic",
"Dust Control" : "Environmental",
"Environmental" : "Environmental",
"Environmental quality" : "Environmental",
"Erosion Control" : "Environmental",
"Evaporation" : "Environmental",
"Evaporative" : "Other",
"Exploration" : "Other",
"Export From Basin" : "Other",
"Export From State" : "Other",
"Exports/Imports" : "Other",
"Federal Reserved" : "Other",
"Federal Reserved Use" : "Other",
"Feed pen operation" : "Agricultural",
"Fire" : "Fire",
"Fire Protection" : "Fire",
"Fire Protection (Instream)" : "Fire",
"Fire Protection From Storage" : "Fire",
"Fire Protection Storage" : "Fire",
"Fish & Wildlife" : "Other",
"Fish and game propogation" : "Fish",
"Fish And Wildlife" : "Fish",
"Fish and Wildlife Preservation and Enhancement" : "Fish",
"Fish Culture" : "Fish",
"Fish Habitat" : "Fish",
"Fish Habitat From Storage" : "Fish",
"Fish Habitat Storage" : "Fish",
"Fish Propagation" : "Fish",
"Fish Propagation From Storage" : "Fish",
"Fish Propagation Storage" : "Fish",
"Fish Raceways" : "Fish",
"Fishery" : "Fish",
"Fishery Enhancement (Instream)" : "Fish",
"Flood Control" : "Flood Control",
"Flood Control Storage" : "Flood Control",
"Flow Augmentation" : "Flood Control",
"Flow Through Fish Pond" : "Flood Control",
"Forest Management" : "Other",
"Frost Protection" : "Environmental",
"GAME PRESERVES" : "Environmental",
"Geothermal" : "Power",
"Geo-Thermal (Heating & Cooling)" : "Heating and Cooling",
"Geothermal boreholes" : "Power",
"Geothermal Heating" : "Heating and Cooling",
"Geo-Thermal(Energy Production)" : "Heating and Cooling",
"Greenhouse" : "Environmental",
"Ground Water Recharge" : "Environmental",
"Ground Water Recharge From Storage" : "Environmental",
"Ground Water Recharge Storage" : "Environmental",
"Groundwater Aquifer Storage & Recovery" : "Storage",
"Groundwater Preservation" : "Storage",
"Groundwater Recharge" : "Storage",
"Group Domestic" : "Domestic",
"Harvesting of Cranberries" : "Agricultural",
"Heat Control" : "Heating and Cooling",
"Heat Exchange" : "Heating and Cooling",
"Heat protection for crops" : "Heating and Cooling",
"Heating" : "Heating and Cooling",
"Heating From Storage" : "Heating and Cooling",
"Heating Storage" : "Heating and Cooling",
"Highway" : "Industrial",
"Highway construction" : "Industrial",
"Household Use Only" : "Domestic",
"Human Consumption" : "Domestic",
"Human Consumption and Livestock" : "Domestic",
"HYDROELECTRIC" : "Power",
"HYDROELECTRIC - PRIORITY" : "Power",
"Incidental Power" : "Power",
"Incidental Underground Storage" : "Storage",
"Individual Irrigation" : "Agricultural",
"Industrial" : "Industrial",
"INDUSTRIAL - POWER GENERATION" : "Industrial",
"INDUSTRIAL - SALT WATER" : "Industrial",
"Industrial From Storage" : "Industrial",
"Industrial Storage" : "Industrial",
"Industrial/Manufacturing Uses" : "Industrial",
"Injection" : "Other",
"Institutional" : "Domestic",
"INSTREAM" : "Instream Flow",
"Instream Flow" : "Instream Flow",
"Intentional Underground Storage" : "Storage",
"Irrigated Agriculture " : "Agricultural",
"Irrigated agriculture_ground" : "Agricultural",
"Irrigated agriculture_reuse" : "Agricultural",
"Irrigated agriculture_surface" : "Agricultural",
"Irrigation" : "Agricultural",
"Irrigation and Domestic" : "Agricultural",
"Irrigation and Incidental Underground Storage" : "Storage",
"Irrigation and Livestock" : "Agricultural",
"Irrigation and Storage (an appropriation approved for both uses)" : "Storage",
"Irrigation from Natural Stream" : "Agricultural",
"Irrigation From Storage" : "Industrial",
"Irrigation Livestock and Domestic" : "Agricultural",
"Irrigation of Cranberries" : "Agricultural",
"Irrigation Storage" : "Agricultural",
"Irrigation Unknown" : "Agricultural",
"Irrigation-Carey Act" : "Agricultural",
"Irrigation-DLE" : "Agricultural",
"Laboratory" : "Other",
"Lake Level Maintenance" : "Environmental",
"Lawn And Garden" : "Domestic",
"Livestock" : "Agricultural",
"Livestock " : "Livestock",
"Livestock and Wildlife" : "Agricultural",
"Livestock_ground" : "Agricultural",
"Livestock_reuse" : "Agricultural",
"Livestock_surface" : "Agricultural",
"Log Deck Sprinkling" : "Industrial",
"M & I" : "Municipal",
"Maintain Level of a Lake" : "Recreation",
"Major and Minor Reservoirs" : "Other",
"Manufacturing" : "Industrial",
"Manufacturing_ground" : "Industrial",
"Manufacturing_reuse" : "Industrial",
"Manufacturing_surface" : "Industrial",
"Meat packing plant" : "Industrial",
"Military - military installations" : "Other",
"Milling" : "Industrial",
"Minerals" : "Other",
"Minimum Stream Flow" : "Environmental",
"Minimum Stream Flow From Storage" : "Environmental",
"Minimum Stream Flow Storage" : "Environmental",
"Minimum Streamflow" : "Environmental",
"Mining" : "Mining",
"Mining and Milling" : "Mining",
"Mining From Storage" : "Mining",
"Mining or milling or oil" : "Mining",
"Mining Storage" : "Mining",
"Mining, Milling, and Dewatering" : "Mining",
"Mining_ground" : "Mining",
"Mining_reuse" : "Mining",
"Mining_surface" : "Mining",
"Mitigation" : "Environmental",
"Mitigation By Delivery" : "Flood Control",
"Mitigation By Delivery From Storage" : "Flood Control",
"Mitigation By Delivery Storage" : "Flood Control",
"Mitigation By NonUse" : "Flood Control",
"Mobile home parks" : "Domestic",
"Monitoring well" : "Other",
"MULTI USE" : "Other",
"Multiple Domestic" : "Domestic",
"Multiple Instream Uses" : "Instream Flow",
"Multiple Purpose" : "Other",
"Multiple Use" : "Other",
"Municipal" : "Municipal",
"Municipal - city or county supplied water" : "Municipal",
"Municipal From Storage" : "Municipal",
"Municipal inter-tie system" : "Other",
"Municipal Storage" : "Municipal",
"Municipal Uses" : "Municipal",
"MUNICIPAL/DOMESTIC AND LIVESTOCK" : "Municipal",
"Municipal_ground" : "Municipal",
"Municipal_reuse" : "Municipal",
"Municipal_surface" : "Municipal",
"Navigation" : "Other",
"Net Effect On River" : "Environmental",
"No pre-1907 water right exists on this land" : "Other",
"No Purpose Identified" : "Unknown",
"No right" : "Unknown",
"No use of right or POD" : "Unknown",
"Non 72-12-1 domestic and livestock watering" : "Other",
"Non 72-12-1 domestic one household" : "Other",
"Non 72-12-1 livestock watering" : "Other",
"Non 72-12-1 multiple domestic households" : "Other",
"NON CONSUMPTIVE" : "Other",
"NON-CONSUMPTIVE" : "Other",
"Non-profit organizational use" : "Other",
"Nursery Uses" : "Agricultural",
"Observation" : "Other",
"Observation And Testing" : "Other",
"Oil field maintenance" : "Industrial",
"Oil production" : "Industrial",
"Oil Well Flooding" : "Industrial",
"Other" : "Other",
"OTHER (FISH HATCHERY)" : "Fish",
"Other Purpose" : "Other",
"OTHER SPILLWAY/FLOODGAT" : "Flood Control",
"Parks and Recreation" : "Recreation",
"Petroleum processing plant" : "Industrial",
"Pollution Abatement" : "Instream Flow",
"Pollution Abatement (Instream)" : "Instream Flow",
"Pollution control well" : "Instream Flow",
"Pond Maintenance" : "Municipal",
"Poultry and egg operation" : "Industrial",
"Power" : "Power",
"Power and Incidental Underground Storage" : "Power",
"Power Development" : "Power",
"Power From Storage" : "Power",
"Power Generation" : "Power",
"Power Generation Nonconsumptive" : "Power",
"POWER GENERATION, NONCONSUMPTIVE" : "Power",
"Power Storage" : "Power",
"Practicably Irrigable Acreage" : "Agricultural",
"Primary and Supplemental Irrigation" : "Agricultural",
"PUBLIC PARKS" : "Recreation",
"Public Supply" : "Domestic",
"Public utility" : "Domestic",
"Quasi-Municipal" : "Municipal",
"Quasi-Municipal Uses" : "Other",
"Railway" : "Industrial",
"Raise Dam (for increase in head for power production)" : "Power",
"Ram" : "Industrial",
"Recharge" : "Recharge",
"Recharge For Irrigation" : "Agricultural",
"Recreation" : "Recreation",
"Recreation - beautification" : "Recreation",
"Recreation Fish Wildlife" : "Fish",
"Recreation From Storage" : "Recreation",
"Recreation Storage" : "Recreation",
"Recreational" : "Recreation",
"Restroom" : "Municipal",
"REUSE" : "Other",
"Riparian Habitat (Instream)" : "Instream Flow",
"Riparian Habitat Restoration Project" : "Instream Flow",
"Riparian Maintenance" : "Instream Flow",
"Road Construction" : "Industrial",
"Rural Water" : "Domestic",
"Sale" : "Commercial",
"Sawmill" : "Industrial",
"School use - public, private, parochial, & universities" : "Municipal",
"Secondary recovery of oil" : "Industrial",
"Sediment Control" : "Environmental",
"See Remarks" : "Other",
"Shop" : "Commercial",
"Snow Making" : "Snow Making",
"Stacked water right" : "Other",
"Steam-Electric Power_ground" : "Power",
"Steam-Electric Power_reuse" : "Power",
"Steam-Electric Power_surface" : "Power",
"Stock" : "Livestock",
"Stock water" : "Livestock",
"Stockpond" : "Other",
"STOCKRAISING" : "Livestock",
"Stockwater" : "Livestock",
"Stockwater From Storage" : "Livestock",
"Stockwater Storage" : "Livestock",
"Stockwatering" : "Livestock",
"Storage" : "Storage",
"Storage and Incidental Underground Storage" : "Storage",
"Storage Use-only (irrigation from reservoir on lands not covered by natural flow appropriation)" : "Storage",
"Storage Use-only for irrigation from reservoir on lands not covered by natural flow appropriation AND for Incidental Underground Storage" : "Storage",
"Strategic water reserve" : "Storage",
"Stream augmentation" : "Flood Control",
"Streamflow Maintenance From Storage" : "Storage",
"Streamflow Maintenance Storage" : "Storage",
"Subdivision" : "Domestic",
"Subirrigation" : "Agricultural",
"Supplemental Cooling (an appropriation for water for cooling through a system that has a prior appropriation for cooling)" : "Heating and Cooling",
"Supplemental Flood Harvesting" : "Flood Control",
"Supplemental Irrigation" : "Agricultural",
"Supplemental Irrigation (irrigation from reservoir on lands also covered by natural flow appropriation.)" : "Agricultural",
"Supplemental Irrigation and Incidental Underground Storage" : "Agricultural",
"Supplemental Power (an appropriation for water for power through a system that has a prior appropriation for power)" : "Power",
"Supplemental Storage (an appropriation that has a prior appropriation for storage)" : "Storage",
"Swimming" : "Recreation",
"Temperature Control" : "Heating and Cooling",
"Temporary Transfer to In-Stream Use" : "Storage",
"Test Well" : "Other",
"To be determined" : "Unknown",
"Transmountain Export" : "Other",
"Trust water, Permanent" : "Other",
"Trust water, Temporary" : "Other",
"Unknown" : "Unknown",
"Unspecified" : "Unknown",
"Use in a Mint Still" : "Industrial",
"Use Within a School" : "Municipal",
"Waste Storage" : "Other",
"Water Fowl" : "Wildlife",
"WATER QUALITY" : "Environmental",
"Water Quality Improvement" : "Environmental",
"Water Quality Improvement From Storage" : "Environmental",
"Water Quality Improvement Storage" : "Environmental",
"Wetlands" : "Environmental",
"Wild And Scenic River" : "Wildlife",
"Wildlife" : "Wildlife",
"Wildlife From Storage" : "Wildlife",
"WILDLIFE MANAGEMENT" : "Wildlife",
"Wildlife refuge" : "Wildlife",
"Wildlife Storage" : "Wildlife",
"Wildlife Waterfowl" : "Wildlife",
"WILDLIFE/WATERFOWL" : "Wildlife"}


def assignWSWCBU(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return np.nan
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = alloWSWCBenUseDict[String1]
        except:
            outList = "Other"
    return outList

dfallo['WSWCBeneficialUse'] = dfallo.apply(lambda row: assignWSWCBU(row['BeneficialUseCV']), axis=1)

In [21]:
#SiteType
SiteTypeCVDic = {"Abandoned" : "Abandoned",
"Aquifer Nnt/Nt Reservation" : "Aquifer",
"Augmentation/Replacement Plan" : "Other",
"canal" : "Canal / Ditch / Stream",
"D/S Limit - Discharge Segment" : "Discharge Point",
"D/S Limit - Diversion Segment" : "Other",
"D/S Limit - Release Segment" : "Other",
"Discharge Point" : "Discharge Point",
"Ditch System" : "Canal / Ditch / Stream",
"ditch" : "Canal / Ditch / Stream",
"Diversion Point - Well" : "Well",
"Diversion Point" : "Diversion Point",
"Diversion" : "Diversion Point",
"drain" : "Drain",
"Effluent" : "Effluent",
"Exchange Plan" : "Exchange Plan",
"Geothermal" : "Other",
"Gravity Other" : "Other",
"Gravity Water-Pump" : "Pump",
"Gravity Well-Pump" : "Well",
"Gravity" : "Gravity",
"ground water collector" : "Groundwater",
"Ground Water" : "Groundwater",
"Groundwater" : "Groundwater",
"headworks gravity flow (or surface water device unknown)" : "Surface Water",
"IBT - Discharge Point" : "Discharge Point",
"IBT - Diversion Point" : "Diversion Point",
"IBT - Off-channel Diversion Pt" : "Diversion Point",
"IBT - Off-channel Reservoir" : "Reservoir",
"IBT - On-channel Reservoir" : "Reservoir",
"IBT - Release Point" : "Other",
"IBT - Return Flow" : "Other",
"irrigation dam" : "Reservoir",
"lake" : "Lake",
"Measuring Point" : "Other",
"Mine" : "Mine",
"Minimum Flow" : "Minimum Flow",
"monitoring well" : "Monitoring Well",
"Off-channel Diversion Point" : "Discharge Point",
"Off-channel Reservoir Complex" : "Reservoir",
"Off-channel Reservoir" : "Reservoir",
"On-channel Reservoir Complex" : "Reservoir",
"On-channel Reservoir" : "Reservoir",
"Other - Rec Use" : "Other",
"Other Ground Water" : "Groundwater",
"Other Surface Water" : "Surface Water",
"Other" : "Other",
"Pipeline" : "Pipeline",
"Point of Rediversion" : "Point of Rediversion",
"Point of Return" : "Point of Return",
"pond" : "Surface Water",
"Power Plant" : "Power Plant",
"Pump" : "Groundwater",
"Reach (Aggregating)" : "Reach",
"Reach" : "Reach",
"Recharge Area Group" : "Recharge Area",
"Recharge Area" : "Recharge Area",
"Release Point" : "Discharge Point",
"reservoir dam" : "Reservoir",
"Reservoir System" : "Reservoir",
"reservoir" : "Reservoir",
"Return Flow Point" : "Point of Return",
"Seep" : "Seep",
"sewage effluent" : "Effluent",
"slough" : "Other",
"spring" : "Spring",
"Storage" : "Storage",
"Stream Gage" : "Other",
"stream" : "Canal / Ditch / Stream",
"sump" : "Sump",
"surface water pump" : "Surface Water",
"Surface Water" : "Surface Water",
"Surface" : "Surface Water",
"Surfacewater_Groundwater" : "Surface & Groundwater",
"U/S Dam" : "Reservoir",
"U/S Limit - Discharge Segment" : "Discharge Point",
"U/S Limit - Diversion Segment" : "Other",
"U/S Limit - Release Segment" : "Other",
"Underground" : "Groundwater",
"Unknown" : "Unknown",
"waste water" : "Effluent",
"Water-Pump Well-Pump" : "Well",
"Water-Pump" : "Pump",
"well (or ground water device unknown)" : "Well",
"Well Field" : "Well",
"Well Group" : "Well",
"well" : "Well",
"Well-Pump Other" : "Pump",
"Well-Pump" : "Well",
"winter runoff" : "winter runoff",
"WWTP Release Point" : "Discharge Point"}
 
def assignSiteType(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "Unknown"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = SiteTypeCVDic[String1]
        except:
            outList = "Unknown"
    return outList

dfallo['SiteTypeCV'] = dfallo.apply(lambda row: assignSiteType(row['SiteTypeCV']), axis=1)

In [22]:
#WaterSourceTypeCV
WaterSourceCVDic = {"Abandoned" : "Other",
"Drain" : "Other",
"Ground Water" : "Groundwater",
"Ground" : "Groundwater",
"Groundwater" : "Groundwater",
"Groundwater/all" : "Groundwater",
"Groundwater/Artesian" : "Groundwater",
"Groundwater/Artesian,Surface Water" : "Surface & Groundwater",
"Groundwater/Dry" : "Groundwater",
"Groundwater/Shallow" : "Groundwater",
"Groundwater/Shallow,Surface Water" : "Surface & Groundwater",
"groundwater/spring" : "Groundwater",
"groundwater/well" : "Groundwater",
"Point of Rediversion" : "Other",
"Point of Return" : "Other",
"reservoir" : "Reservoir / Storage",
"Reuse" : "Other",
"storage" : "Reservoir / Storage",
"Subsurface" : "Other",
"Surface Water" : "Surface Water",
"Surface Water" : "Surface Water",
"Surface" : "Surface Water",
"Surfacewater" : "Surface Water",
"Unknown" : "Unknown",
"Unspecified" : "Unknown"}
 
def assignWaterSourceType(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "Unknown"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = WaterSourceCVDic[String1]
        except:
            outList = "Unknown"
    return outList

dfallo['WaterSourceTypeCV'] = dfallo.apply(lambda row: assignWaterSourceType(row['WaterSourceTypeCV']), axis=1)

In [23]:
#AllocationOwner
AllocationOwnerDic = {
"DREAMWORK MANAGEMENT INC; U.S.-FOREST SERVICE; NEL, ALLEN G." : "US Forest Service",
"FOREST SERVICE LINE SPG" : "US Forest Service",
"FOREST SERVICE SPRING PIPELINE" : "US Forest Service",
"FOREST SERVICE,UNITED STATES" : "US Forest Service",
"FOREST SERVICE,UNITED STATES OF AMERICA" : "US Forest Service",
"Forest Service,United States of America -" : "US Forest Service",
"FOREST SERVICE,USDA" : "US Forest Service",
"MAGDALENA RANGER DISTRICT,US Forest Service" : "US Forest Service",
"N.D. Forest Service" : "US Forest Service",
"Rowser and et al.,Lillian C.,USA Forest Service,Rasmussen,Roger J." : "US Forest Service",
"Rowser and et al.,Lillian C.,USA Forest Service,Rasmusson,Roger J.,Hood,E. Ward & Linnea" : "US Forest Service",
"SANTA FE NATIONAL FOREST,USDA FOREST SERVICE" : "US Forest Service",
"State of Utah Division of Wildlife Resources,USA Forest Service" : "US Forest Service",
"Steed,Newel,USA Forest Service" : "US Forest Service",
"TEXAS A&M FOREST SERVICE" : "US Forest Service",
"U S FOREST SERVICE" : "US Forest Service",
"U S Forest Service" : "US Forest Service",
"U. S. FOREST SERVICE" : "US Forest Service",
"U. S. Forest Service, Lassen National Forest, Hat Creek Ranger District" : "US Forest Service",
"U.S. Forest Service" : "US Forest Service",
"U.S. FOREST SERVICE" : "US Forest Service",
"U.S. FOREST SERVICE" : "US Forest Service",
"U.S. FOREST SERVICE; APPLEGATE RANGER DISTRICT" : "US Forest Service",
"U.S. FOREST SERVICE; BEND RANGER DISTRICT" : "US Forest Service",
"U.S. FOREST SERVICE; COLUMBIA RIVER GORGE SCENIC AREA" : "US Forest Service",
"U.S. FOREST SERVICE; MIDDLE FORK RANGER DISTRICT" : "US Forest Service",
"U.S.-FOREST SERVICE" : "US Forest Service",
"U.S.-FOREST SERVICE; U.S.-AGRICULTURE DEPARTMENT" : "US Forest Service",
"UNITED STATES FOREST SERVICE" : "US Forest Service",
"UNITED STATES FOREST SERVICE" : "US Forest Service",
"United States Of America Acting Through Usda Forest Service" : "US Forest Service",
"UNITED STATES OF AMERICA FOREST SERVICE" : "US Forest Service",
"UNITED STATES OF AMERICA, FOREST SERVICE" : "US Forest Service",
"US Forest Service" : "US Forest Service",
"US Forest Service" : "US Forest Service",
"US Forest Service Mount Baker," : "US Forest Service",
"US Forest Service NW International," : "US Forest Service",
"US Forest Service Snoqualmie," : "US Forest Service",
"US Forest Service Wenatchee," : "US Forest Service",
"US Forest Service," : "US Forest Service",
"US Forest Service, LOS PADRES NATIONAL FOREST" : "US Forest Service",
"USA FOREST SERVICE" : "US Forest Service",
"USA Forest Service" : "US Forest Service",
"USA Forest Service - Dixie National Forest" : "US Forest Service",
"USA Forest Service - Dixie National Service" : "US Forest Service",
"USA Forest Service - Fishlake National Forest" : "US Forest Service",
"USA Forest Service / Fishlake National Forest" : "US Forest Service",
"USA Forest Service,Ackerman,Wilford" : "US Forest Service",
"USA Forest Service,Dixie National Forest,USA FOREST SERVICE" : "US Forest Service",
"USA Forest Service,Gardner,Alan Dean,Gardner,Dean C.,Burgess,C. Judd,Gardner,Larry H." : "US Forest Service",
"USA Forest Service,Heirs of E. J. Hancock" : "US Forest Service",
"USA Forest Service,Jones,Richard N." : "US Forest Service",
"USA Forest Service,USA Forest Service 2" : "US Forest Service",
"USA Forest Service,USA Forestt Service" : "US Forest Service",
"USA Forest Service,USA Forsest Service" : "US Forest Service",
"USA Forest Service." : "US Forest Service",
"USA National Forest 2,USA Forest Service" : "US Forest Service",
"USA USDA FOREST SERVICE" : "US Forest Service",
"USDA FOREST SERVICE" : "US Forest Service",
"USDA Forest Service" : "US Forest Service",
"USDA FOREST SERVICE CIBOLA NF" : "US Forest Service",
"USDA FOREST SERVICE LAKE TAHOE BASIN MANAGEMENT UNIT" : "US Forest Service",
"USDA FOREST SERVICE(CARSON NF)" : "US Forest Service",
"USDA Forest Service," : "US Forest Service",
"USDA FOREST SERVICE, SEQUOIA NATIONAL FOREST" : "US Forest Service",
"USDA Forest Service, USA" : "US Forest Service",
"USDA FOREST SERVICE,CARSON NF" : "US Forest Service",
"USDA FOREST SERVICE; DESCHUTES NATIONAL FOREST" : "US Forest Service",
"USDA FOREST SERVICES" : "US Forest Service",
"USDA, FOREST SERVICE,CARSON NF" : "US Forest Service",
"West Panguitch Irrigation and Reservoir Company,USA Forest Service" : "US Forest Service",
"Department of Energy,United States of America" : "US Department of Energy",
"M-K Ferguson Co. for USA Department of Energy" : "US Department of Energy",
"MK-Ferguson Co. for the USA Department of Energy" : "US Department of Energy",
"U S DEPARTMENT OF ENERGY" : "US Department of Energy",
"U.S. Department of Energy" : "US Department of Energy",
"U.S.-DEPARTMENT OF ENERGY,  YUCCA MOUNTAIN SITE CHARACTERIZATION PROJECT" : "US Department of Energy",
"U.S-DEPARTMENT OF ENERGY, YUCCA MOUNTAIN SITE CHARACTERIZATION PROJECT" : "US Department of Energy",
"UNITED STATES DEPARTMENT OF ENERGY" : "US Department of Energy",
"United States of America,U.S. Department of Energy" : "US Department of Energy",
"US Department of Energy," : "US Department of Energy",
"USA Department of Energy Grand Junction Projects" : "US Department of Energy",
"Bankhead,S. Clair,UT Reclamation Mitigation and Conservation Comm,United States of America Bureau of Reclamation,Corbridge,William B." : "US Bureau Reclamation",
"Bryan, Timothy E & Jennifer,Fletcher, Gary & Carla,Teanaway Associates,,Perkins, Charles J & Catherine,US Bureau Reclamation - Yakima," : "US Bureau Reclamation",
"Bureau Of Reclamation" : "US Bureau Reclamation",
"BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"Bureau Of Reclamation," : "US Bureau Reclamation",
"Bureau of Reclamation, Provo Area Office,United States of America,Richardson,Floyd C.,Richardson,W. Earl,Richardson,Jay V.,Sandy City,Richardson,Duane G." : "US Bureau Reclamation",
"Bureau of Reclamation,The Secretary of the Interior" : "US Bureau Reclamation",
"Bureau of Reclamation,United State of America" : "US Bureau Reclamation",
"BUREAU OF RECLAMATION,UNITED STATES D.O.I." : "US Bureau Reclamation",
"Bureau of Reclamation,United States of America" : "US Bureau Reclamation",
"Bureau of Reclamation,United States of America," : "US Bureau Reclamation",
"Bureau of Reclamation,United States of America,Corbridge,William B." : "US Bureau Reclamation",
"BUREAU OF RECLAMATION; UMATILLA FIELD OFFICE" : "US Bureau Reclamation",
"Heart K Ranch,,US Bureau Reclamation - Yakima," : "US Bureau Reclamation",
"Hirst,Paul and Bonnie,Weber River Water Users Association,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Lambert,Paul,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Morgan County,Weber River Water Users` Assocation,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Provo Area Office,U.S.A. Bureau of Reclamation" : "US Bureau Reclamation",
"Provo Area Office,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Remund,Glen & Unice,United States of America Bureau of Reclamation" : "US Bureau Reclamation",
"Smith,Clarence,Siddoway,Benjamin and Beryl,Vernon,James,Vernon,Francis,Gines,Mark E. and Marianne A.,USA Bureau of Reclamation,Smith,Dick,Siddoway,Alan Siddoway and LouDean,Crandall Creek Irrigation Company" : "US Bureau Reclamation",
"U S BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U. S. DEPT. OF THE INTERIOR,BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S. BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S. Bureau Of Reclamation" : "US Bureau Reclamation",
"U.S. BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S. BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"U.S.A. Bureau of Reclamation" : "US Bureau Reclamation",
"U.S.A. Bureau of Reclamation,Fitzgerald,Jackson R." : "US Bureau Reclamation",
"U.S.-BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"United States Bureau of Reclamation" : "US Bureau Reclamation",
"United States of America Bureau of Reclamation,Spencer Todd and Kelly Colleen Adamson Family Trust" : "US Bureau Reclamation",
"United States of America, Bureau of Reclamation" : "US Bureau Reclamation",
"US BUREAU OF RECLAMATION" : "US Bureau Reclamation",
"US Bureau of Reclamation" : "US Bureau Reclamation",
"US Bureau of Reclamation - Boise," : "US Bureau Reclamation",
"US Bureau Of Reclamation," : "US Bureau Reclamation",
"USA Bureau of Reclamation" : "US Bureau Reclamation",
"USA Bureau of Reclamation -- Provo Area Office" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Boyden,John" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Ivie,James" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Miller,Max" : "US Bureau Reclamation",
"USA Bureau of Reclamation,Weber River Water Users Association,Sanders Properties" : "US Bureau Reclamation",
"UT Reclamation Mitigation and Conservation Comm,United States of America Bureau of Reclamation,Bankhead,S. Clair,Corbridge,William B" : "US Bureau Reclamation",
"Weber River Water Users` Association,Morgan County,USA Bureau of Reclamation" : "US Bureau Reclamation",
"Wilde Revocable Trust,Debra A.,Wilde Land and Livestock Company,Wilde `A` Ditch Company,Faddis,Andrew,Wilde,Henry,Wilson,John,Wilde,Richard & Betty,USA Bureau of Reclamation,Beech,Thomas L.,Wilde Revocable Trust,David Elliot,Wilde,Mary" : "US Bureau Reclamation",
"Wilkinson,Allen,United States of America Bureau of Reclamation" : "US Bureau Reclamation",
"Allen,T. L.,USA Bureau of Reclamation,Eldredge,Edmund,Robinson,Steve B. and Julie Ann,North Chalk Creek Irrigation Company" : "US Bureau Reclamation",
" - Cedar City Field Office,US Bureau of Land Management" : "US Bureau of Land Management",
" - Cedar City Field Office,US Department of Interior, Bureau of Land Management" : "US Bureau of Land Management",
" - Cedar City Field Office,US Department of the Interior, Bureau of Land Management" : "US Bureau of Land Management",
" - Cedar City Field Office,USDI Bureau of Land Management" : "US Bureau of Land Management",
"#NAME?,US Department of Interior, Bureau of Land Management" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"Bureau of Land Management" : "US Bureau of Land Management",
"Bureau of Land Management," : "US Bureau of Land Management",
"Bureau of Land Management, Bakersfield" : "US Bureau of Land Management",
"Bureau of Land Management,Box Elder County" : "US Bureau of Land Management",
"Bureau of Land Management,Davis,Matthew W Davis (II) and Shawnee,Tebbs Living Trust,Richard Ray and Irene H." : "US Bureau of Land Management",
"Bureau of Land Management,Flinspach,Richard and Colleen" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,SOCORRO FIELD OFFICE" : "US Bureau of Land Management",
"Bureau of Land Management,Stuntz Valley Ranch LC" : "US Bureau of Land Management",
"Bureau of Land Management,T Y Cattle Company" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,UNITED STATES" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,UNITED STATES GOVERNMENT" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,UNITED STATES OF AMERICA" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,US DEPARTMENT OF INTERIOR" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,US DEPT LAND MANAGEMENT" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,US DEPT OF INTERIOR" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT,USDI" : "US Bureau of Land Management",
"BUREAU OF LAND MANAGEMENT; OWYHEE FIELD OFFICE" : "US Bureau of Land Management",
"Carbon County,USA Bureau of Land Management" : "US Bureau of Land Management",
"Cedar City Field Office,USDI Bureau of Land Management;" : "US Bureau of Land Management",
"Chew Livestock Inc,Bureau of Land Management" : "US Bureau of Land Management",
"Department of Interior Bureau of Land Management" : "US Bureau of Land Management",
"Department of Natural Resources,State of Utah,,USA Bureau of Land Management" : "US Bureau of Land Management",
"Division of Wildlife Resources,State of Utah,,USA Bureau of Land Management,Price Field Office" : "US Bureau of Land Management",
"Gemstone Mining Inc.,USA Bureau of Land Management" : "US Bureau of Land Management",
"Grouse Creek Ranch LC,USA Bureau of Land Management" : "US Bureau of Land Management",
"Jordan,W. Clift,USA Bureau of Land Management" : "US Bureau of Land Management",
"Materials Energy Research and Recovery,USA Bureau of Land Management" : "US Bureau of Land Management",
"Moab District USA Bureau of Land Management" : "US Bureau of Land Management",
"Nelson,Rick M.,USA Bureau of Land Management" : "US Bureau of Land Management",
"NORTHERN WASHOE RANCHING MANAGEMENT, LLC; DUCK LAKE RANCH, LLC; U.S.-BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"NORTHERN WASHOE RANCHING MANAGEMENT, LLC; U.S.-BUREAU OF LAND MANAGEMENT; DUCK LAKE RANCH, LLC" : "US Bureau of Land Management",
"Richfield District USA Bureau of Land Management" : "US Bureau of Land Management",
"Saint George Field Office,USDI Bureau of Land Management" : "US Bureau of Land Management",
"Saint George Field Office,USDI Bureau of Land Management;" : "US Bureau of Land Management",
"Skull Valley Company Ltd.,USA Bureau of Land Management" : "US Bureau of Land Management",
"Staker,James Allen,USA Bureau of Land Management" : "US Bureau of Land Management",
"State of Utah Division of Wildlife Resources,USA Bureau of Land Management" : "US Bureau of Land Management",
"Thousand Peaks Ranches, Inc.,USA Bureau of Land Management,Richfield District" : "US Bureau of Land Management",
"U S BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S. Bureau Of Land Management" : "US Bureau of Land Management",
"U.S. BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S. DEPARTMENT OF THE INTERIOR; BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S.-BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"U.S.-BUREAU OF LAND MANAGEMENT; BRIGHT-HOLLAND CORPORATION" : "US Bureau of Land Management",
"U.S.-BUREAU OF LAND MANAGEMENT; WHITE PINE LUMBER CO." : "US Bureau of Land Management",
"U.S.D.I., Bureau of Land Management" : "US Bureau of Land Management",
"United States Of America Acting Through The Department Of Interior Bureau Of Land Management" : "US Bureau of Land Management",
"United States of America Bureau of Land Management,Fillmore District" : "US Bureau of Land Management",
"UNITED STATES OF AMERICA, DEPARTMENT OF THE INTERIOR, BUREAU OF LAND MANAGEMENT; WULFENSTEIN TRUST R-501; SPRING MOUNTAIN MOTORSPORTS RANCH; PAHRUMP GATEWAY UTILITY; SPRING MOUNTAIN RACEWAY LLC" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT / BARSTOW FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT / PALM SPRINGS FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT, ALTURAS FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT, RIDGECREST FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT, UKIAH FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT/ ARCATA FIELD OFFICE" : "US Bureau of Land Management",
"US BUREAU OF LAND MANAGEMENT/NEEDLES FIELD OFFICE" : "US Bureau of Land Management",
"US DEPT OF THE INTERIOR,BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"USA Bureau of Land Management" : "US Bureau of Land Management",
"USA Bureau of Land Management - Filmore Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management (Richfield District)" : "US Bureau of Land Management",
"USA Bureau of Land Management Vernal District" : "US Bureau of Land Management",
"USA Bureau of Land Management, Richfield District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Cedar City District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Fillmore District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Fillmore Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management,Kanab District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Kanab Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management,Moab District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Monticello  District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Monticello District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Ogden District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Price District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Price Field Office" : "US Bureau of Land Management",
"USA Bureau of Land Management,Richfield District" : "US Bureau of Land Management",
"USA Bureau of Land Management,Vernal District" : "US Bureau of Land Management",
"USDI BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"USDI BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"Utah School and Institutional Trust Lands Admin.,USA Bureau of Land Management" : "US Bureau of Land Management",
"West Panguitch Irrigation and Reservoir Company,State of Utah Board of Water Resources,USA Bureau of Land Management" : "US Bureau of Land Management",
"WHITE PINE LUMBER CO; U.S.-BUREAU OF LAND MANAGEMENT" : "US Bureau of Land Management",
"ARMY CAMP SPRING" : "US Army",
"CORPS OF ENGRS. ABIQUIU DAM,US ARMY ENGR DIST ALBQ." : "US Army",
"Cummins, Dale F,US Army Corps Of Engineers," : "US Army",
"Dept Of The Army Corps Of Engineers," : "US Army",
"LONGHORN ARMY AMMUNITION PLANT,U.S. DEPARTMENT OF THE INTERIOR FISH AND WILDLIFE SERVICE" : "US Army",
"U S ARMY CORPS OF ENGINEERS" : "US Army",
"U S Army Corps of Engineers" : "US Army",
"U S Army, Department of" : "US Army",
"U.S Army Corps Of Engineers" : "US Army",
"U.S. ARMY CORP OF ENGINEERS" : "US Army",
"U.S. Army Corps Of Engineers" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS SEATTLE" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS WALLA WALLA" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS, WALLA WALLA DISTRICT" : "US Army",
"U.S. ARMY CORPS OF ENGINEERS; PORTLAND DISTRICT" : "US Army",
"U.S. ARMY CORPS OF ENGINEERSSEATTLE" : "US Army",
"U.S. ARMY CORPS OF ENGINEERSWALLA WALLA" : "US Army",
"U.S. Army Dugway Proving Ground" : "US Army",
"U.S. ARMY WHITE SANDS MISSILE" : "US Army",
"U.S. ARMY-WHITE SANDS MISSILE" : "US Army",
"UNITED STATES ARMY CORPS OF ENGINEERS" : "US Army",
"US ARMY" : "US Army",
"Us Army Corps Of Engineers" : "US Army",
"US ARMY CORPS OF ENGINEERS" : "US Army",
"US Army Corps Of Engineers," : "US Army",
"US ARMY CORPS OF ENGINGEERS" : "US Army",
"US DEPARTMENT OF THE ARMY" : "US Army",
"USA Army Dugway Proving Ground" : "US Army",
"USA Army Dugway Proving Grounds" : "US Army",
"USA Army Engineer District Corps of Engineers" : "US Army",
"USA Army for Dugway Proving Ground" : "US Army",
"USA Army Ogden Sub-Depot of Tooele Ordinance Depot" : "US Army",
"USA Army Tooele Army Depot South" : "US Army",
"USA Department of the Army" : "US Army",
"USA Department of the Army - Dugway Proving Ground" : "US Army",
"USA Department of the Army - Tooele Army Depot" : "US Army",
"USA Department of the Army Arsenal" : "US Army",
"USARMY Corp Of Engineers," : "US Army",
"USARMY Corps Engineers Columbia Cnty," : "US Army",
"USARMY Corps Engineers Seattle," : "US Army",
"USARMY Corps Engineers Walla Walla," : "US Army",
"USARMY Corps Engineers," : "US Army",
"USARMY Corps Engineers,,USDA Agricultural Research Service WRPIS," : "US Army",
"WA Fish & Wildlife Dept - ERO,,USARMY Corps Engineers," : "US Army",
"Camelot International Resort Corporation,United States of America" : "United States of America",
"Christensen,Mark B.,Fish and Wildlife Service,United States of America" : "United States of America",
"CIBOLA NATIONAL FOREST,UNITED STATES OF AMERICA," : "United States of America",
"Ekker,Sheena,Utah School and Institutional Trust Lands Admin.,Hodges,Jimmy,McArthur,Ty,United States of America" : "United States of America",
"Fish and Wildlife Service,United States of America" : "United States of America",
"Fish and Wildlife Service,United States of America," : "United States of America",
"INTERMOUNTAIN LAND CO.; MOONROCKS GATEWAY LLC; UNITED STATES OF AMERICA" : "United States of America",
"LAMB, SHELDON W. & ISABEL H.; UNITED STATES OF AMERICA" : "United States of America",
"Llewellyn,Robert W.,United States of America" : "United States of America",
"R. CLYDE HIIBEL FAMILY TRUST; UNITED STATES OF AMERICA" : "United States of America",
"The United States of America" : "United States of America",
"THE UNITED STATES OF AMERICA IN TRUST FOR THE PYRAMID LAKE PAIUTE TRIBE; GARAVANTA LAND AND LIVESTOCK COMPANY SUCCESSOR TO J. GARAVANTA" : "United States of America",
"THOMAS, DARRELL E.; THOMAS, BEVERLY J.; UNITED STATES OF AMERICA" : "United States of America",
"United States Of America" : "United States of America",
"UNITED STATES OF AMERICA" : "United States of America",
"UNITED STATES OF AMERICA" : "United States of America",
"United States of America" : "United States of America",
"UNITED STATES OF AMERICA (BLM)" : "United States of America",
"United States Of America Acting Through" : "United States of America",
"United States of America in trust for Raymond Murray,Burton,John Ira" : "United States of America",
"UNITED STATES OF AMERICA IN TRUST FOR SUCH SHOSHONE INDIANS SOUTHERN NEVADA" : "United States of America",
"UNITED STATES OF AMERICA IN TRUST FOR THE WASHOE TRIBE OF INDIANS" : "United States of America",
"UNITED STATES OF AMERICA IN TRUST FOR YOMBA TRIBE OF SHOSHONE INDIANS" : "United States of America",
"UNITED STATES OF AMERICA NAS" : "United States of America",
"UNITED STATES OF AMERICA NAVAL AIR STATION FALLON" : "United States of America",
"United States Of America Trustee" : "United States of America",
"United States Of America," : "United States of America",
"UNITED STATES OF AMERICA,  C/O COMMANDER AND DISTRICT ENGINEER,   UNITED STATES ARMY ENGINEER  DISTRICT, LOUISVILLE" : "United States of America",
"United States of America, Utah" : "United States of America",
"United States of America,BBRD, LC,Ottesen,Nellie" : "United States of America",
"United States of America,Owen Probst and Rosetta B. Probst Trust" : "United States of America",
"United States of America,The,Conger,Wm. B. and Florance H" : "United States of America",
"United States of America,Virgin River Land Preservation Inc." : "United States of America",
"UNITED STATES OF AMERICA; STREETS; UNITED STATES OF AMERICA IN TRUST FOR THE RENO SPARKS INDIAN COLONY; NEVADA-DEPARTMENT OF TRANSPORTATION; SIERRA PACIFIC POWER COMPANY; RENO SPARKS INDIAN COLONY; RENO LODGE #13 F AND A M" : "United States of America",
"United States of American" : "United States of America",
"USA-United States Of America" : "United States of America",
"Utah Reclamation Mitigation and Conservation Commission,United States Of America" : "United States of America",
"WASHOE COUNTY; PRATT, TOM & LANNETTE; UNITED STATES OF AMERICA" : "United States of America",
"WS-BUCKLAND, LLC; UNITED STATES OF AMERICA" : "United States of America"}
 
def assignAllocationOwner(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "Other"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = AllocationOwnerDic[String1]
        except:
            outList = "Other"
    return outList

dfallo['AllocationOwner'] = dfallo.apply(lambda row: assignAllocationOwner(row['AllocationOwner']), axis=1)

In [24]:
#Sort by AllocationAmountID and SiteUUID
dfallo = dfallo.sort_values(by=['AllocationAmountID', 'SiteUUID'], ascending=True).reset_index()
dfallo

,index,AllocationAmountID,AllocationNativeID,AllocationAmount,AllocationMaximum,IrrigatedAcreage,AllocationPriorityDateID,PrimaryUseCategoryCV,AllocationOwner,BeneficialUseCV,...,SiteTypeCV,Longitude,Latitude,WaterSourceID,WaterSourceName,WaterSourceTypeCV,OrganizationID,OrganizationUUID,State,WSWCBeneficialUse
0,151368,2734281,50769.493780000004-0-C-100503,150.0,0.0,0.0,1446,Irrigation,Other,Unknown,...,Unknown,-104.468875,40.379142,38325,SOUTH PLATTE RIVER,Surface Water,1,CODWR,CO,Unknown
1,151369,2734282,51356.0-0-C-100503,120.0,0.0,0.0,1984,Irrigation,Other,Storage,...,Unknown,-104.468875,40.379142,38325,SOUTH PLATTE RIVER,Surface Water,1,CODWR,CO,Storage
2,151370,2734283,53330.0-0-C-100503,1000.0,0.0,0.0,14095,Irrigation,Other,Unknown,...,Unknown,-104.468875,40.379142,38325,SOUTH PLATTE RIVER,Surface Water,1,CODWR,CO,Unknown
3,151371,2734284,55637.0-0-C-100503,358.0,0.0,0.0,5202,Irrigation,Other,Unknown,...,Unknown,-104.468875,40.379142,38325,SOUTH PLATTE RIVER,Surface Water,1,CODWR,CO,Unknown
4,151372,2734285,55989.0-0-C-100503,1080.0,0.0,0.0,6100,Irrigation,Other,Unknown,...,Unknown,-104.468875,40.379142,38325,SOUTH PLATTE RIVER,Surface Water,1,CODWR,CO,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048418,729052,4713858,C1497,0.0,0.0,0.0,44037,Irrigation,Other,Irrigation,...,Diversion Point,-100.023706,30.489069,559475,Unspecified,Unknown,26,TCEQ,TX,Agricultural
1048419,728031,4713859,C3439,0.0,0.0,0.0,44037,Irrigation,Other,Irrigation,...,Diversion Point,-95.861930,28.845659,559475,Unspecified,Unknown,26,TCEQ,TX,Agricultural
1048420,730036,4713860,C1861,0.0,0.0,0.0,44037,Irrigation,Other,Irrigation,...,Diversion Point,-98.918007,31.154508,559475,Unspecified,Unknown,26,TCEQ,TX,Agricultural
1048421,730056,4713861,C3786,0.0,0.0,0.0,44037,Irrigation,Other,Irrigation,...,Diversion Point,-101.176992,35.887029,559475,Unspecified,Unknown,26,TCEQ,TX,Agricultural


In [25]:
## Create Sites Dataframe - with attached Allo and Benuse info ##
########################################
outdfallo = pd.DataFrame()

#The Columns
outdfallo['AA_ID'] = dfallo['AllocationAmountID'].astype(str) 
outdfallo['SiteUUID'] = dfallo['SiteUUID'].astype(str)
outdfallo['SN_ID'] = dfallo['AllocationNativeID'].astype(str) 
outdfallo['Lat'] = dfallo['Latitude'].astype(float) 
outdfallo['Long'] = dfallo['Longitude'].astype(float)
outdfallo['WBenUse'] = dfallo['WSWCBeneficialUse'].astype(str)
outdfallo['State'] = dfallo['State'].astype(str) 
outdfallo['AA_CFS'] = dfallo['AllocationAmount'].astype(float) 
outdfallo['AA_AF'] = dfallo['IrrigatedAcreage'].astype(float) 
outdfallo['AAM_AF'] = dfallo['AllocationMaximum'].astype(float) 
outdfallo['PD'] = dfallo['Date']
outdfallo['WSN'] = dfallo['WaterSourceName'].astype(str)
outdfallo['SiteTypeCV'] = dfallo['SiteTypeCV'].astype(str)
outdfallo['WaterSourceTypeCV'] = dfallo['WaterSourceTypeCV'].astype(str)
outdfallo['AllocationOwner'] = dfallo['AllocationOwner'].astype(str)

outdfallo

,AA_ID,SiteUUID,SN_ID,Lat,Long,WBenUse,State,AA_CFS,AA_AF,AAM_AF,PD,WSN,SiteTypeCV,WaterSourceTypeCV,AllocationOwner
0,2734281,CO_2,50769.493780000004-0-C-100503,40.379142,-104.468875,Unknown,CO,150.0,0.0,0.0,1985-03-11,SOUTH PLATTE RIVER,Unknown,Surface Water,Other
1,2734282,CO_2,51356.0-0-C-100503,40.379142,-104.468875,Storage,CO,120.0,0.0,0.0,1990-08-10,SOUTH PLATTE RIVER,Unknown,Surface Water,Other
2,2734283,CO_2,53330.0-0-C-100503,40.379142,-104.468875,Unknown,CO,1000.0,0.0,0.0,1996-01-05,SOUTH PLATTE RIVER,Unknown,Surface Water,Other
3,2734284,CO_2,55637.0-0-C-100503,40.379142,-104.468875,Unknown,CO,358.0,0.0,0.0,2002-04-30,SOUTH PLATTE RIVER,Unknown,Surface Water,Other
4,2734285,CO_2,55989.0-0-C-100503,40.379142,-104.468875,Unknown,CO,1080.0,0.0,0.0,2003-04-17,SOUTH PLATTE RIVER,Unknown,Surface Water,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048418,4713858,TX_3465,C1497,30.489069,-100.023706,Agricultural,TX,0.0,0.0,0.0,2020-06-24,Unspecified,Diversion Point,Unknown,Other
1048419,4713859,TX_275,C3439,28.845659,-95.861930,Agricultural,TX,0.0,0.0,0.0,2020-06-24,Unspecified,Diversion Point,Unknown,Other
1048420,4713860,TX_4224,C1861,31.154508,-98.918007,Agricultural,TX,0.0,0.0,0.0,2020-06-24,Unspecified,Diversion Point,Unknown,Other
1048421,4713861,TX_4240,C3786,35.887029,-101.176992,Agricultural,TX,0.0,0.0,0.0,2020-06-24,Unspecified,Diversion Point,Unknown,Other


In [26]:
#Exporting Outbound DataFrame to working csv file.
outdfallo.to_csv('P_AllowLJSite.csv', index=False)  # The output